# 1) Import the necessary packages

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
import numpy as np
import pandas as pd
from scipy import stats
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set(color_codes=True)
from collections import Counter,namedtuple,defaultdict,OrderedDict
import warnings
warnings.filterwarnings('ignore')

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [3]:
from numpy import ones

#### Checking bytes for numpy

In [4]:
int64arr = ones((1024, 1024), dtype=np.uint64)
int16arr=ones((1024,1024),dtype=np.uint16)
int64arr.nbytes

8388608

In [5]:
int16arr.nbytes

2097152

# 2) Loading the Admission Table 

In [6]:
data_AdmissionsCorePopulatedTable=pd.read_csv("/content/drive/My Drive/Colab Notebooks/AdmissionsCorePopulatedTable.txt",sep='\t')
data_AdmissionsCorePopulatedTable.head(10)

,PatientID,AdmissionID,AdmissionStartDate,AdmissionEndDate
0,9380F9E3-1927-42F3-9731-03A74D4E4C6B,5,2011-03-23 00:14:52.237,2011-03-28 01:57:56.467
1,0A89658C-C739-45CA-9BF1-CBDDDFB922C0,1,1974-02-10 00:04:28.357,1974-02-16 21:04:43.333
2,0A89658C-C739-45CA-9BF1-CBDDDFB922C0,2,1991-05-22 15:08:52.743,1991-05-29 12:30:08.737
3,0A89658C-C739-45CA-9BF1-CBDDDFB922C0,3,1995-02-26 08:01:26.477,1995-02-28 16:45:04.407
4,0A89658C-C739-45CA-9BF1-CBDDDFB922C0,4,2005-03-17 04:51:15.747,2005-04-04 04:04:14.160
5,0A89658C-C739-45CA-9BF1-CBDDDFB922C0,5,2008-04-12 23:17:33.217,2008-04-16 03:22:35.737
6,EB1C36B1-0697-4145-95EE-0B0C44943339,1,1995-12-04 16:01:07.700,1995-12-17 22:59:46.223
7,EB1C36B1-0697-4145-95EE-0B0C44943339,2,2001-08-02 05:05:13.547,2001-08-16 02:35:48.323
8,EB1C36B1-0697-4145-95EE-0B0C44943339,3,2009-11-15 09:31:16.750,2009-12-01 06:49:13.053
9,F5F37898-D271-4D68-9C0E-18B1184411C6,1,1989-03-15 16:32:19.170,1989-03-19 07:04:55.640


# 3) Adding a new column " AdmissionDuration" in this table

In [7]:
data_AdmissionsCorePopulatedTable['AdmissionEndDate']=pd.to_datetime(data_AdmissionsCorePopulatedTable['AdmissionEndDate']).dt.date

In [8]:
data_AdmissionsCorePopulatedTable['AdmissionStartDate']=pd.to_datetime(data_AdmissionsCorePopulatedTable['AdmissionStartDate']).dt.date

In [9]:
data_AdmissionsCorePopulatedTable.head(5)

,PatientID,AdmissionID,AdmissionStartDate,AdmissionEndDate
0,9380F9E3-1927-42F3-9731-03A74D4E4C6B,5,2011-03-23,2011-03-28
1,0A89658C-C739-45CA-9BF1-CBDDDFB922C0,1,1974-02-10,1974-02-16
2,0A89658C-C739-45CA-9BF1-CBDDDFB922C0,2,1991-05-22,1991-05-29
3,0A89658C-C739-45CA-9BF1-CBDDDFB922C0,3,1995-02-26,1995-02-28
4,0A89658C-C739-45CA-9BF1-CBDDDFB922C0,4,2005-03-17,2005-04-04


In [10]:
data_AdmissionsCorePopulatedTable['AdmissionDuration']=(data_AdmissionsCorePopulatedTable['AdmissionEndDate']-data_AdmissionsCorePopulatedTable['AdmissionStartDate'])

In [11]:
data_AdmissionsCorePopulatedTable.head(10)

,PatientID,AdmissionID,AdmissionStartDate,AdmissionEndDate,AdmissionDuration
0,9380F9E3-1927-42F3-9731-03A74D4E4C6B,5,2011-03-23,2011-03-28,5 days
1,0A89658C-C739-45CA-9BF1-CBDDDFB922C0,1,1974-02-10,1974-02-16,6 days
2,0A89658C-C739-45CA-9BF1-CBDDDFB922C0,2,1991-05-22,1991-05-29,7 days
3,0A89658C-C739-45CA-9BF1-CBDDDFB922C0,3,1995-02-26,1995-02-28,2 days
4,0A89658C-C739-45CA-9BF1-CBDDDFB922C0,4,2005-03-17,2005-04-04,18 days
5,0A89658C-C739-45CA-9BF1-CBDDDFB922C0,5,2008-04-12,2008-04-16,4 days
6,EB1C36B1-0697-4145-95EE-0B0C44943339,1,1995-12-04,1995-12-17,13 days
7,EB1C36B1-0697-4145-95EE-0B0C44943339,2,2001-08-02,2001-08-16,14 days
8,EB1C36B1-0697-4145-95EE-0B0C44943339,3,2009-11-15,2009-12-01,16 days
9,F5F37898-D271-4D68-9C0E-18B1184411C6,1,1989-03-15,1989-03-19,4 days


In [12]:
data_AdmissionsCorePopulatedTable.shape

(361760, 5)

# 4) Check for the duplicate rows in the column and drop if necessary

In [13]:
data_AdmissionsCorePopulatedTable.duplicated(subset=["PatientID","AdmissionID","AdmissionDuration"],keep='first')

0         False
1         False
2         False
3         False
4         False
          ...  
361755    False
361756    False
361757    False
361758    False
361759    False
Length: 361760, dtype: bool

In [14]:
data_AdmissionsCorePopulatedTable.PatientID.count()

361760

In [15]:
data_AdmissionsCorePopulatedTable=data_AdmissionsCorePopulatedTable.drop_duplicates(subset=["PatientID","AdmissionID","AdmissionDuration"],keep='first')

In [16]:
data_AdmissionsCorePopulatedTable.PatientID.count()

361760

In [17]:
data_AdmissionsCorePopulatedTable=data_AdmissionsCorePopulatedTable.drop(columns=['AdmissionStartDate','AdmissionEndDate'],axis=1)
data_AdmissionsCorePopulatedTable.head()

,PatientID,AdmissionID,AdmissionDuration
0,9380F9E3-1927-42F3-9731-03A74D4E4C6B,5,5 days
1,0A89658C-C739-45CA-9BF1-CBDDDFB922C0,1,6 days
2,0A89658C-C739-45CA-9BF1-CBDDDFB922C0,2,7 days
3,0A89658C-C739-45CA-9BF1-CBDDDFB922C0,3,2 days
4,0A89658C-C739-45CA-9BF1-CBDDDFB922C0,4,18 days


In [18]:
data_AdmissionsCorePopulatedTable.PatientID.unique()

array(['9380F9E3-1927-42F3-9731-03A74D4E4C6B',
       '0A89658C-C739-45CA-9BF1-CBDDDFB922C0',
       'EB1C36B1-0697-4145-95EE-0B0C44943339', ...,
       'B90062A9-4D85-4A9E-A493-4F2AFFBB8808',
       'DB495784-37B2-4185-8D66-B76E8374099D',
       'B7A09A1C-3648-4E28-A2A2-6B202E3BC39C'], dtype=object)

In [19]:
data_AdmissionsCorePopulatedTable.shape

(361760, 3)

In [20]:
data_AdmissionsCorePopulatedTable.size

1085280

In [21]:
data_AdmissionsCorePopulatedTable.isna().sum().to_frame().T

,PatientID,AdmissionID,AdmissionDuration
0,0,0,0


# 5)Loading the Admission Diagnosis Table

In [22]:
data_AdmissionsDiagnosesCorePopulatedTable=pd.read_csv("/content/drive/My Drive/Colab Notebooks/AdmissionsDiagnosesCorePopulatedTable.txt",sep='\t')
data_AdmissionsDiagnosesCorePopulatedTable.head(6)

,PatientID,AdmissionID,PrimaryDiagnosisCode,PrimaryDiagnosisDescription
0,54C6E968-45B3-46B1-A64F-2CE3124F2A65,3,F80.1,Expressive language disorder
1,54C6E968-45B3-46B1-A64F-2CE3124F2A65,4,R04.81,Acute idiopathic pulmonary hemorrhage in infants
2,54C6E968-45B3-46B1-A64F-2CE3124F2A65,5,I36,Nonrheumatic tricuspid valve disorders
3,9DD23357-9BEB-43E4-802D-1AB7ACDD4A3A,1,H16.43,Localized vascularization of cornea
4,9DD23357-9BEB-43E4-802D-1AB7ACDD4A3A,2,M05.161,Rheumatoid lung disease with rheumatoid arthri...
5,9DD23357-9BEB-43E4-802D-1AB7ACDD4A3A,3,M06.011,Rheumatoid arthritis without rheumatoid factor...


In [23]:
data_AdmissionsDiagnosesCorePopulatedTable.shape

(361760, 4)

In [24]:
data_AdmissionsDiagnosesCorePopulatedTable.PatientID.count()

361760

# 6) Check for the duplicate rows and drop if necessary

In [25]:
data_AdmissionsDiagnosesCorePopulatedTable.duplicated(subset=["PatientID","AdmissionID","PrimaryDiagnosisCode","PrimaryDiagnosisDescription"],keep='first')

0         False
1         False
2         False
3         False
4         False
          ...  
361755    False
361756    False
361757    False
361758    False
361759    False
Length: 361760, dtype: bool

In [26]:
data_AdmissionsDiagnosesCorePopulatedTable=data_AdmissionsDiagnosesCorePopulatedTable.drop_duplicates(subset=["PatientID","AdmissionID","PrimaryDiagnosisCode","PrimaryDiagnosisDescription"],keep='first')

In [27]:
data_AdmissionsDiagnosesCorePopulatedTable.PatientID.count()

361760

In [28]:
data_AdmissionsDiagnosesCorePopulatedTable.shape

(361760, 4)

In [29]:
data_AdmissionsDiagnosesCorePopulatedTable.size

1447040

In [30]:
data_AdmissionsDiagnosesCorePopulatedTable.isna().sum().to_frame().T

,PatientID,AdmissionID,PrimaryDiagnosisCode,PrimaryDiagnosisDescription
0,0,0,0,0


# 7) Loading the Patient Table

In [31]:
data_PatientCorePopulatedTable=pd.read_csv("/content/drive/My Drive/Colab Notebooks/PatientCorePopulatedTable.txt",sep="\t")
data_PatientCorePopulatedTable.head(6)

,PatientID,PatientGender,PatientDateOfBirth,PatientRace,PatientMaritalStatus,PatientLanguage,PatientPopulationPercentageBelowPoverty
0,F7CF0FE9-AFCD-49EF-BFB3-E42302FFA0D3,Female,1951-07-10 07:29:47.293,Asian,Single,English,13.70
1,C3935FBC-DBBA-4844-BBE4-A175FA508454,Male,1956-01-27 22:46:39.380,African American,Single,English,15.73
2,1CA33F6F-2E84-4C99-AF6A-D40F7B4DB27F,Male,1972-12-22 10:11:01.867,White,Married,English,7.09
3,81606388-2471-42A4-A6F1-1868AE25CFC3,Male,1984-01-17 00:49:06.903,Asian,Separated,Spanish,2.17
4,E3120DE9-3361-40CF-A618-265C769E75A2,Female,1978-12-21 07:24:08.957,White,Married,English,18.67
5,5C043111-3F94-44BC-A889-97D44ACCC7F6,Female,1974-09-25 18:38:02.440,African American,Married,English,2.57


In [32]:
data_PatientCorePopulatedTable.shape

(100000, 7)

# 8) Check for the duplicate rows in this table and drop if this necessary

In [33]:
data_PatientCorePopulatedTable.duplicated(subset=['PatientID','PatientGender','PatientMaritalStatus','PatientDateOfBirth'],keep='first')

0        False
1        False
2        False
3        False
4        False
         ...  
99995    False
99996    False
99997    False
99998    False
99999    False
Length: 100000, dtype: bool

In [34]:
data_PatientCorePopulatedTable.PatientID.count()

100000

In [35]:
data_PatientCorePopulatedTable=data_PatientCorePopulatedTable.drop_duplicates(subset=['PatientID','PatientGender','PatientMaritalStatus','PatientDateOfBirth'],keep='first')

In [36]:
data_PatientCorePopulatedTable.PatientID.count()

100000

# 9) Removing the columns not required for model training

In [37]:
data_PatientCorePopulatedTable=data_PatientCorePopulatedTable.drop(columns=['PatientLanguage','PatientPopulationPercentageBelowPoverty'],axis=1)
data_PatientCorePopulatedTable.head()

,PatientID,PatientGender,PatientDateOfBirth,PatientRace,PatientMaritalStatus
0,F7CF0FE9-AFCD-49EF-BFB3-E42302FFA0D3,Female,1951-07-10 07:29:47.293,Asian,Single
1,C3935FBC-DBBA-4844-BBE4-A175FA508454,Male,1956-01-27 22:46:39.380,African American,Single
2,1CA33F6F-2E84-4C99-AF6A-D40F7B4DB27F,Male,1972-12-22 10:11:01.867,White,Married
3,81606388-2471-42A4-A6F1-1868AE25CFC3,Male,1984-01-17 00:49:06.903,Asian,Separated
4,E3120DE9-3361-40CF-A618-265C769E75A2,Female,1978-12-21 07:24:08.957,White,Married


In [38]:
data_PatientCorePopulatedTable.PatientID.count()

100000

In [39]:
data_PatientCorePopulatedTable.shape

(100000, 5)

In [40]:
data_PatientCorePopulatedTable.size

500000

In [41]:
data_PatientCorePopulatedTable.isna().sum().to_frame().T

,PatientID,PatientGender,PatientDateOfBirth,PatientRace,PatientMaritalStatus
0,0,0,0,0,0


# 10) Merging Admission Core table and Admission Diagnosis table to generate Admission Group

In [42]:
data_AdmissionsCorePopulatedTable.dtypes

PatientID                     object
AdmissionID                    int64
AdmissionDuration    timedelta64[ns]
dtype: object

In [43]:
data_AdmissionsDiagnosesCorePopulatedTable.dtypes

PatientID                      object
AdmissionID                     int64
PrimaryDiagnosisCode           object
PrimaryDiagnosisDescription    object
dtype: object

In [44]:
data_Admissions_Core_Diagnoses_Merge=data_AdmissionsCorePopulatedTable.merge(data_AdmissionsDiagnosesCorePopulatedTable,how='outer',on=['PatientID','AdmissionID'])
data_Admissions_Core_Diagnoses_Merge.head()

,PatientID,AdmissionID,AdmissionDuration,PrimaryDiagnosisCode,PrimaryDiagnosisDescription
0,9380F9E3-1927-42F3-9731-03A74D4E4C6B,5,5 days,M11.07,"Hydroxyapatite deposition disease, ankle and foot"
1,0A89658C-C739-45CA-9BF1-CBDDDFB922C0,1,6 days,C13.2,Malignant neoplasm of posterior wall of hypoph...
2,0A89658C-C739-45CA-9BF1-CBDDDFB922C0,2,7 days,G30,Alzheimer's disease
3,0A89658C-C739-45CA-9BF1-CBDDDFB922C0,3,2 days,H67.2,"Otitis media in diseases classified elsewhere,..."
4,0A89658C-C739-45CA-9BF1-CBDDDFB922C0,4,18 days,M50.1,Cervical disc disorder with radiculopathy


In [45]:
data_Admissions_Core_Diagnoses_Merge.shape

(361760, 5)

In [46]:
data_Admissions_Core_Diagnoses_Merge

,PatientID,AdmissionID,AdmissionDuration,PrimaryDiagnosisCode,PrimaryDiagnosisDescription
0,9380F9E3-1927-42F3-9731-03A74D4E4C6B,5,5 days,M11.07,"Hydroxyapatite deposition disease, ankle and foot"
1,0A89658C-C739-45CA-9BF1-CBDDDFB922C0,1,6 days,C13.2,Malignant neoplasm of posterior wall of hypoph...
2,0A89658C-C739-45CA-9BF1-CBDDDFB922C0,2,7 days,G30,Alzheimer's disease
3,0A89658C-C739-45CA-9BF1-CBDDDFB922C0,3,2 days,H67.2,"Otitis media in diseases classified elsewhere,..."
4,0A89658C-C739-45CA-9BF1-CBDDDFB922C0,4,18 days,M50.1,Cervical disc disorder with radiculopathy
...,...,...,...,...,...
361755,DB495784-37B2-4185-8D66-B76E8374099D,5,16 days,M90.631,"Osteitis deformans in neoplastic diseases, rig..."
361756,DB495784-37B2-4185-8D66-B76E8374099D,6,2 days,E09.21,Drug or chemical induced diabetes mellitus wit...
361757,B7A09A1C-3648-4E28-A2A2-6B202E3BC39C,1,12 days,C57.22,Malignant neoplasm of left round ligament
361758,B7A09A1C-3648-4E28-A2A2-6B202E3BC39C,2,12 days,C92.12,"Chronic myeloid leukemia, BCR/ABL-positive, in..."


In [47]:
data_Admissions_Core_Diagnoses_Merge.PatientID.count()

361760

In [48]:

data_Admissions_Core_Diagnoses_Merge.shape

(361760, 5)

In [49]:
data_Admissions_Core_Diagnoses_Merge.size

1808800

In [50]:
data_Admissions_Core_Diagnoses_Merge.isna().sum().to_frame().T

,PatientID,AdmissionID,AdmissionDuration,PrimaryDiagnosisCode,PrimaryDiagnosisDescription
0,0,0,0,0,0


# 11) Merging Patient table with Admission group to generate Patient group

In [51]:
data_Patients_Core_Admission_Core_Diagnoses_Merge=pd.merge(data_Admissions_Core_Diagnoses_Merge,data_PatientCorePopulatedTable,on='PatientID',how='outer')
data_Patients_Core_Admission_Core_Diagnoses_Merge.head()

,PatientID,AdmissionID,AdmissionDuration,PrimaryDiagnosisCode,PrimaryDiagnosisDescription,PatientGender,PatientDateOfBirth,PatientRace,PatientMaritalStatus
0,9380F9E3-1927-42F3-9731-03A74D4E4C6B,5,5 days,M11.07,"Hydroxyapatite deposition disease, ankle and foot",Male,1930-08-12 10:59:03.917,Unknown,Divorced
1,9380F9E3-1927-42F3-9731-03A74D4E4C6B,1,19 days,D37.030,Neoplasm of uncertain behavior of the parotid ...,Male,1930-08-12 10:59:03.917,Unknown,Divorced
2,9380F9E3-1927-42F3-9731-03A74D4E4C6B,2,13 days,F31.61,"Bipolar disorder, current episode mixed, mild",Male,1930-08-12 10:59:03.917,Unknown,Divorced
3,9380F9E3-1927-42F3-9731-03A74D4E4C6B,3,13 days,T82.21,Mechanical complication of coronary artery byp...,Male,1930-08-12 10:59:03.917,Unknown,Divorced
4,9380F9E3-1927-42F3-9731-03A74D4E4C6B,4,3 days,C10.2,Malignant neoplasm of lateral wall of oropharynx,Male,1930-08-12 10:59:03.917,Unknown,Divorced


In [52]:
data_Patients_Core_Admission_Core_Diagnoses_Merge.PatientID.count()

361760

In [53]:
data_Patients_Core_Admission_Core_Diagnoses_Merge.shape

(361760, 9)

In [54]:

data_Patients_Core_Admission_Core_Diagnoses_Merge.size

3255840

In [55]:
data_Patients_Core_Admission_Core_Diagnoses_Merge.isna().sum().to_frame().T

,PatientID,AdmissionID,AdmissionDuration,PrimaryDiagnosisCode,PrimaryDiagnosisDescription,PatientGender,PatientDateOfBirth,PatientRace,PatientMaritalStatus
0,0,0,0,0,0,0,0,0,0


# 12) Loading the Lab table

In [56]:
data_LabsCorePopulatedTable=pd.read_csv("/content/drive/My Drive/Colab Notebooks/LabsCore_EHR_TransformedTable.txt",sep='\t')
data_LabsCorePopulatedTable.head(2)

,PatientID,CBC: WHITE BLOOD CELL COUNT,CBC: RED BLOOD CELL COUNT,CBC: HEMOGLOBIN,CBC: HEMATOCRIT,CBC: MEAN CORPUSCULAR VOLUME,CBC: MCH,CBC: MCHC,CBC: RDW,CBC: PLATELET COUNT,CBC: ABSOLUTE NEUTROPHILS,CBC: ABSOLUTE LYMPHOCYTES,CBC: NEUTROPHILS,CBC: LYMPHOCYTES,CBC: MONOCYTES,CBC: EOSINOPHILS,CBC: BASOPHILS,METABOLIC: SODIUM,METABOLIC: POTASSIUM,METABOLIC: CHLORIDE,METABOLIC: CARBON DIOXIDE,METABOLIC: ANION GAP,METABOLIC: GLUCOSE,METABOLIC: BUN,METABOLIC: CREATININE,METABOLIC: TOTAL PROTEIN,METABOLIC: ALBUMIN,METABOLIC: CALCIUM,METABOLIC: BILI TOTAL,METABOLIC: AST/SGOT,METABOLIC: ALT/SGPT,METABOLIC: ALK PHOS,URINALYSIS: SPECIFIC GRAVITY,URINALYSIS: PH,URINALYSIS: RED BLOOD CELLS,URINALYSIS: WHITE BLOOD CELLS
0,56016E96-25C9-4FC8-9426-A8A411C73066,8.3@@k/cumm,3.8@@m/cumm,14.5@@gm/dl,35.3@@%,72.6@@fl,37.6@@pg,34.6@@g/dl,9.3@@%,344.9@@k/cumm,60.4@@%,20@@%,7.5@@k/cumm,2.1@@k/cumm,0.9@@k/cumm,0.3@@k/cumm,0.2@@k/cumm,134.5@@mmol/L,4.6@@mmol/L,113.3@@mmol/L,35.9@@mmol/L,9.7@@mmol/L,131.3@@mg/dL,23.7@@mg/dL,1.1@@mg/dL,9.4@@gm/dL,4@@gm/dL,11@@mg/dL,0.3@@mg/dL,38.9@@U/L,60.8@@U/L,77@@U/L,1@@no unit,7@@no unit,2.9@@rbc/hpf,4.9@@wbc/hpf
1,06AC4F93-BC23-48E0-870E-C2E7752C7EE6,8@@k/cumm,5.7@@m/cumm,14.5@@gm/dl,48.3@@%,83.5@@fl,26.9@@pg,37.1@@g/dl,11.2@@%,413.4@@k/cumm,64@@%,29.4@@%,7.9@@k/cumm,4.3@@k/cumm,0.7@@k/cumm,0.3@@k/cumm,0.1@@k/cumm,132.5@@mmol/L,5.9@@mmol/L,111.7@@mmol/L,22.8@@mmol/L,4.8@@mmol/L,110.3@@mg/dL,15.8@@mg/dL,0.7@@mg/dL,9.3@@gm/dL,5@@gm/dL,7.6@@mg/dL,0.9@@mg/dL,37.9@@U/L,51.6@@U/L,117.3@@U/L,1@@no unit,5.1@@no unit,1.8@@rbc/hpf,4.9@@wbc/hpf


In [57]:
data_LabsCorePopulatedTable.shape

(100001, 36)

In [58]:
data_LabsCorePopulatedTable.PatientID.count()

100001

In [59]:
data_LabsCorePopulatedTable.shape

(100001, 36)

In [60]:
data_LabsCorePopulatedTable.size

3600036

In [61]:
data_LabsCorePopulatedTable.isna().sum().to_frame().T

,PatientID,CBC: WHITE BLOOD CELL COUNT,CBC: RED BLOOD CELL COUNT,CBC: HEMOGLOBIN,CBC: HEMATOCRIT,CBC: MEAN CORPUSCULAR VOLUME,CBC: MCH,CBC: MCHC,CBC: RDW,CBC: PLATELET COUNT,CBC: ABSOLUTE NEUTROPHILS,CBC: ABSOLUTE LYMPHOCYTES,CBC: NEUTROPHILS,CBC: LYMPHOCYTES,CBC: MONOCYTES,CBC: EOSINOPHILS,CBC: BASOPHILS,METABOLIC: SODIUM,METABOLIC: POTASSIUM,METABOLIC: CHLORIDE,METABOLIC: CARBON DIOXIDE,METABOLIC: ANION GAP,METABOLIC: GLUCOSE,METABOLIC: BUN,METABOLIC: CREATININE,METABOLIC: TOTAL PROTEIN,METABOLIC: ALBUMIN,METABOLIC: CALCIUM,METABOLIC: BILI TOTAL,METABOLIC: AST/SGOT,METABOLIC: ALT/SGPT,METABOLIC: ALK PHOS,URINALYSIS: SPECIFIC GRAVITY,URINALYSIS: PH,URINALYSIS: RED BLOOD CELLS,URINALYSIS: WHITE BLOOD CELLS
0,0,15,11,12,4,12,11,7,7,11,5,7,7,5,7,10,8,10,12,9,11,6,12,9,11,8,11,10,9,8,10,4,12,7,8,4


# 13) Splitting the column of LabValue of each lab using split

In [62]:
a=['CBC: WHITE BLOOD CELL COUNT','CBC: RED BLOOD CELL COUNT', 'CBC: HEMOGLOBIN', 'CBC: HEMATOCRIT',
       'CBC: MEAN CORPUSCULAR VOLUME', 'CBC: MCH', 'CBC: MCHC', 'CBC: RDW',
       'CBC: PLATELET COUNT', 'CBC: ABSOLUTE NEUTROPHILS',
       'CBC: ABSOLUTE LYMPHOCYTES', 'CBC: NEUTROPHILS', 'CBC: LYMPHOCYTES',
       'CBC: MONOCYTES', 'CBC: EOSINOPHILS', 'CBC: BASOPHILS',
       'METABOLIC: SODIUM', 'METABOLIC: POTASSIUM', 'METABOLIC: CHLORIDE',
       'METABOLIC: CARBON DIOXIDE', 'METABOLIC: ANION GAP',
       'METABOLIC: GLUCOSE', 'METABOLIC: BUN', 'METABOLIC: CREATININE',
       'METABOLIC: TOTAL PROTEIN', 'METABOLIC: ALBUMIN', 'METABOLIC: CALCIUM',
       'METABOLIC: BILI TOTAL', 'METABOLIC: AST/SGOT', 'METABOLIC: ALT/SGPT',
       'METABOLIC: ALK PHOS', 'URINALYSIS: SPECIFIC GRAVITY', 'URINALYSIS: PH',
       'URINALYSIS: RED BLOOD CELLS', 'URINALYSIS: WHITE BLOOD CELLS']

for i in a:

    data_LabsCorePopulatedTable[i]= data_LabsCorePopulatedTable[i].str.split("@@", n = 1, expand = True) 

In [63]:
data_LabsCorePopulatedTable.head(5)

,PatientID,CBC: WHITE BLOOD CELL COUNT,CBC: RED BLOOD CELL COUNT,CBC: HEMOGLOBIN,CBC: HEMATOCRIT,CBC: MEAN CORPUSCULAR VOLUME,CBC: MCH,CBC: MCHC,CBC: RDW,CBC: PLATELET COUNT,CBC: ABSOLUTE NEUTROPHILS,CBC: ABSOLUTE LYMPHOCYTES,CBC: NEUTROPHILS,CBC: LYMPHOCYTES,CBC: MONOCYTES,CBC: EOSINOPHILS,CBC: BASOPHILS,METABOLIC: SODIUM,METABOLIC: POTASSIUM,METABOLIC: CHLORIDE,METABOLIC: CARBON DIOXIDE,METABOLIC: ANION GAP,METABOLIC: GLUCOSE,METABOLIC: BUN,METABOLIC: CREATININE,METABOLIC: TOTAL PROTEIN,METABOLIC: ALBUMIN,METABOLIC: CALCIUM,METABOLIC: BILI TOTAL,METABOLIC: AST/SGOT,METABOLIC: ALT/SGPT,METABOLIC: ALK PHOS,URINALYSIS: SPECIFIC GRAVITY,URINALYSIS: PH,URINALYSIS: RED BLOOD CELLS,URINALYSIS: WHITE BLOOD CELLS
0,56016E96-25C9-4FC8-9426-A8A411C73066,8.3,3.8,14.5,35.3,72.6,37.6,34.6,9.3,344.9,60.4,20,7.5,2.1,0.9,0.3,0.2,134.5,4.6,113.3,35.9,9.7,131.3,23.7,1.1,9.4,4,11,0.3,38.9,60.8,77,1,7,2.9,4.9
1,06AC4F93-BC23-48E0-870E-C2E7752C7EE6,8,5.7,14.5,48.3,83.5,26.9,37.1,11.2,413.4,64,29.4,7.9,4.3,0.7,0.3,0.1,132.5,5.9,111.7,22.8,4.8,110.3,15.8,0.7,9.3,5,7.6,0.9,37.9,51.6,117.3,1,5.1,1.8,4.9
2,B03638F1-09E2-45E6-9B20-9EA6076DAD4C,9.7,6.5,11.7,45.8,92,37.5,36.6,15,214,70,32.2,6.3,4.3,0.7,0.3,0,141,3.8,90.2,18.5,8,90,24.5,1.1,8.4,4.6,7.6,1.2,18.4,65.6,87.6,1,4.8,2.3,1.6
3,B2A32C66-54FC-4B15-9C54-4AF7ADA207FC,4.3,4.5,18.3,30.9,85.6,39.8,36.4,11.3,137.1,69,20.2,1.2,2.4,1.1,0.3,0,134.5,3.9,112.9,19.3,15.1,70.2,14.1,0.8,9.2,3.1,10.8,0.8,15.2,42.2,62.5,1,4.8,1,4.6
4,CDF2AF96-252B-49AB-8DEA-74CD1E910AD1,4,6.6,11.9,47.3,76.5,27.9,38.1,10.3,436.7,71.2,15.8,10.7,2.9,0.6,0.3,0.1,127.7,4.6,91.4,32.2,17,90.7,17.7,0.8,6.7,4.4,7.4,0,17.7,60.1,98.4,1,5.3,1.4,4.5


# 14) Merging Lab table with Patient group to generate Lab group

In [64]:
data=pd.merge(data_Patients_Core_Admission_Core_Diagnoses_Merge,data_LabsCorePopulatedTable,on=['PatientID'],how='inner')
data.head(10)

,PatientID,AdmissionID,AdmissionDuration,PrimaryDiagnosisCode,PrimaryDiagnosisDescription,PatientGender,PatientDateOfBirth,PatientRace,PatientMaritalStatus,CBC: WHITE BLOOD CELL COUNT,CBC: RED BLOOD CELL COUNT,CBC: HEMOGLOBIN,CBC: HEMATOCRIT,CBC: MEAN CORPUSCULAR VOLUME,CBC: MCH,CBC: MCHC,CBC: RDW,CBC: PLATELET COUNT,CBC: ABSOLUTE NEUTROPHILS,CBC: ABSOLUTE LYMPHOCYTES,CBC: NEUTROPHILS,CBC: LYMPHOCYTES,CBC: MONOCYTES,CBC: EOSINOPHILS,CBC: BASOPHILS,METABOLIC: SODIUM,METABOLIC: POTASSIUM,METABOLIC: CHLORIDE,METABOLIC: CARBON DIOXIDE,METABOLIC: ANION GAP,METABOLIC: GLUCOSE,METABOLIC: BUN,METABOLIC: CREATININE,METABOLIC: TOTAL PROTEIN,METABOLIC: ALBUMIN,METABOLIC: CALCIUM,METABOLIC: BILI TOTAL,METABOLIC: AST/SGOT,METABOLIC: ALT/SGPT,METABOLIC: ALK PHOS,URINALYSIS: SPECIFIC GRAVITY,URINALYSIS: PH,URINALYSIS: RED BLOOD CELLS,URINALYSIS: WHITE BLOOD CELLS
0,9380F9E3-1927-42F3-9731-03A74D4E4C6B,5,5 days,M11.07,"Hydroxyapatite deposition disease, ankle and foot",Male,1930-08-12 10:59:03.917,Unknown,Divorced,9.4,5.1,13.9,35.5,93.6,26.9,35.4,16.3,294.9,71.5,32.3,5.4,3.3,0.4,0.1,0.2,133.5,3.8,92.9,20.1,17.7,114.2,10,1,5.5,3.3,8.6,0,41.8,54.1,62,1,5.7,0.7,5.8
1,9380F9E3-1927-42F3-9731-03A74D4E4C6B,1,19 days,D37.030,Neoplasm of uncertain behavior of the parotid ...,Male,1930-08-12 10:59:03.917,Unknown,Divorced,9.4,5.1,13.9,35.5,93.6,26.9,35.4,16.3,294.9,71.5,32.3,5.4,3.3,0.4,0.1,0.2,133.5,3.8,92.9,20.1,17.7,114.2,10,1,5.5,3.3,8.6,0,41.8,54.1,62,1,5.7,0.7,5.8
2,9380F9E3-1927-42F3-9731-03A74D4E4C6B,2,13 days,F31.61,"Bipolar disorder, current episode mixed, mild",Male,1930-08-12 10:59:03.917,Unknown,Divorced,9.4,5.1,13.9,35.5,93.6,26.9,35.4,16.3,294.9,71.5,32.3,5.4,3.3,0.4,0.1,0.2,133.5,3.8,92.9,20.1,17.7,114.2,10,1,5.5,3.3,8.6,0,41.8,54.1,62,1,5.7,0.7,5.8
3,9380F9E3-1927-42F3-9731-03A74D4E4C6B,3,13 days,T82.21,Mechanical complication of coronary artery byp...,Male,1930-08-12 10:59:03.917,Unknown,Divorced,9.4,5.1,13.9,35.5,93.6,26.9,35.4,16.3,294.9,71.5,32.3,5.4,3.3,0.4,0.1,0.2,133.5,3.8,92.9,20.1,17.7,114.2,10,1,5.5,3.3,8.6,0,41.8,54.1,62,1,5.7,0.7,5.8
4,9380F9E3-1927-42F3-9731-03A74D4E4C6B,4,3 days,C10.2,Malignant neoplasm of lateral wall of oropharynx,Male,1930-08-12 10:59:03.917,Unknown,Divorced,9.4,5.1,13.9,35.5,93.6,26.9,35.4,16.3,294.9,71.5,32.3,5.4,3.3,0.4,0.1,0.2,133.5,3.8,92.9,20.1,17.7,114.2,10,1,5.5,3.3,8.6,0,41.8,54.1,62,1,5.7,0.7,5.8
5,0A89658C-C739-45CA-9BF1-CBDDDFB922C0,1,6 days,C13.2,Malignant neoplasm of posterior wall of hypoph...,Male,1949-03-06 16:15:08.950,African American,Single,10.6,6.4,16.2,48.6,77.7,23.2,38.7,13.6,428.1,70,25.3,8.3,3.4,0.6,0.4,0.1,151.9,4.7,93.9,19.9,11.4,88.1,17.9,1.1,8.1,2.9,8.5,0.7,29.1,25.8,71,1,6.2,1.3,1.4
6,0A89658C-C739-45CA-9BF1-CBDDDFB922C0,2,7 days,G30,Alzheimer's disease,Male,1949-03-06 16:15:08.950,African American,Single,10.6,6.4,16.2,48.6,77.7,23.2,38.7,13.6,428.1,70,25.3,8.3,3.4,0.6,0.4,0.1,151.9,4.7,93.9,19.9,11.4,88.1,17.9,1.1,8.1,2.9,8.5,0.7,29.1,25.8,71,1,6.2,1.3,1.4
7,0A89658C-C739-45CA-9BF1-CBDDDFB922C0,3,2 days,H67.2,"Otitis media in diseases classified elsewhere,...",Male,1949-03-06 16:15:08.950,African American,Single,10.6,6.4,16.2,48.6,77.7,23.2,38.7,13.6,428.1,70,25.3,8.3,3.4,0.6,0.4,0.1,151.9,4.7,93.9,19.9,11.4,88.1,17.9,1.1,8.1,2.9,8.5,0.7,29.1,25.8,71,1,6.2,1.3,1.4
8,0A89658C-C739-45CA-9BF1-CBDDDFB922C0,4,18 days,M50.1,Cervical disc disorder with radiculopathy,Male,1949-03-06 16:15:08.950,African American,Single,10.6,6.4,16.2,48.6,77.7,23.2,38.7,13.6,428.1,70,25.3,8.3,3.4,0.6,0.4,0.1,151.9,4.7,93.9,19.9,11.4,88.1,17.9,1.1,8.1,2.9,8.5,0.7,29.1,25.8,71,1,6.2,1.3,1.4
9,0A89658C-C739-45CA-9BF1-CBDDDFB922C0,5,4 days,C79.11,Secondary malignant neoplasm of bladder,Male,1949-03-06 16:15:08.950,African American,Single,10.6,6.4,16.2,48.6,77.7,23.2,38.7,13.6,428.1,70,25.3,8.3,3.4,0.6,0.4,0.1,151.9,4.7,93.9,19.9,11.4,88.1,17.9,1.1,8.1,2.9,8.5,0.7,29.1,25.8,71,1,6.2,1.3,1.4


In [65]:
data.shape

(361760, 44)

In [66]:
data.PatientID.count()

361760

In [67]:
data.size

15917440

In [68]:
data.columns

Index(['PatientID', 'AdmissionID', 'AdmissionDuration', 'PrimaryDiagnosisCode',
       'PrimaryDiagnosisDescription', 'PatientGender', 'PatientDateOfBirth',
       'PatientRace', 'PatientMaritalStatus', 'CBC: WHITE BLOOD CELL COUNT',
       'CBC: RED BLOOD CELL COUNT', 'CBC: HEMOGLOBIN', 'CBC: HEMATOCRIT',
       'CBC: MEAN CORPUSCULAR VOLUME', 'CBC: MCH', 'CBC: MCHC', 'CBC: RDW',
       'CBC: PLATELET COUNT', 'CBC: ABSOLUTE NEUTROPHILS',
       'CBC: ABSOLUTE LYMPHOCYTES', 'CBC: NEUTROPHILS', 'CBC: LYMPHOCYTES',
       'CBC: MONOCYTES', 'CBC: EOSINOPHILS', 'CBC: BASOPHILS',
       'METABOLIC: SODIUM', 'METABOLIC: POTASSIUM', 'METABOLIC: CHLORIDE',
       'METABOLIC: CARBON DIOXIDE', 'METABOLIC: ANION GAP',
       'METABOLIC: GLUCOSE', 'METABOLIC: BUN', 'METABOLIC: CREATININE',
       'METABOLIC: TOTAL PROTEIN', 'METABOLIC: ALBUMIN', 'METABOLIC: CALCIUM',
       'METABOLIC: BILI TOTAL', 'METABOLIC: AST/SGOT', 'METABOLIC: ALT/SGPT',
       'METABOLIC: ALK PHOS', 'URINALYSIS: SPECIFIC GR

# 15) Encoding the lab values from different LabName columns

In [69]:
data.columns

Index(['PatientID', 'AdmissionID', 'AdmissionDuration', 'PrimaryDiagnosisCode',
       'PrimaryDiagnosisDescription', 'PatientGender', 'PatientDateOfBirth',
       'PatientRace', 'PatientMaritalStatus', 'CBC: WHITE BLOOD CELL COUNT',
       'CBC: RED BLOOD CELL COUNT', 'CBC: HEMOGLOBIN', 'CBC: HEMATOCRIT',
       'CBC: MEAN CORPUSCULAR VOLUME', 'CBC: MCH', 'CBC: MCHC', 'CBC: RDW',
       'CBC: PLATELET COUNT', 'CBC: ABSOLUTE NEUTROPHILS',
       'CBC: ABSOLUTE LYMPHOCYTES', 'CBC: NEUTROPHILS', 'CBC: LYMPHOCYTES',
       'CBC: MONOCYTES', 'CBC: EOSINOPHILS', 'CBC: BASOPHILS',
       'METABOLIC: SODIUM', 'METABOLIC: POTASSIUM', 'METABOLIC: CHLORIDE',
       'METABOLIC: CARBON DIOXIDE', 'METABOLIC: ANION GAP',
       'METABOLIC: GLUCOSE', 'METABOLIC: BUN', 'METABOLIC: CREATININE',
       'METABOLIC: TOTAL PROTEIN', 'METABOLIC: ALBUMIN', 'METABOLIC: CALCIUM',
       'METABOLIC: BILI TOTAL', 'METABOLIC: AST/SGOT', 'METABOLIC: ALT/SGPT',
       'METABOLIC: ALK PHOS', 'URINALYSIS: SPECIFIC GR

In [70]:
data.dtypes

PatientID                                 object
AdmissionID                                int64
AdmissionDuration                timedelta64[ns]
PrimaryDiagnosisCode                      object
PrimaryDiagnosisDescription               object
PatientGender                             object
PatientDateOfBirth                        object
PatientRace                               object
PatientMaritalStatus                      object
CBC: WHITE BLOOD CELL COUNT               object
CBC: RED BLOOD CELL COUNT                 object
CBC: HEMOGLOBIN                           object
CBC: HEMATOCRIT                           object
CBC: MEAN CORPUSCULAR VOLUME              object
CBC: MCH                                  object
CBC: MCHC                                 object
CBC: RDW                                  object
CBC: PLATELET COUNT                       object
CBC: ABSOLUTE NEUTROPHILS                 object
CBC: ABSOLUTE LYMPHOCYTES                 object
CBC: NEUTROPHILS    

# 16) Generating Patient Age and Patient Age category columns in Lab group

In [71]:
import datetime
from datetime import date as dt

In [72]:

dt.today()

datetime.date(2020, 9, 2)

In [73]:
type(data['PatientDateOfBirth'])

pandas.core.series.Series

In [74]:
data['PatientDateOfBirth'] = pd.to_datetime(data['PatientDateOfBirth']).dt.date

In [75]:
data.tail()

,PatientID,AdmissionID,AdmissionDuration,PrimaryDiagnosisCode,PrimaryDiagnosisDescription,PatientGender,PatientDateOfBirth,PatientRace,PatientMaritalStatus,CBC: WHITE BLOOD CELL COUNT,CBC: RED BLOOD CELL COUNT,CBC: HEMOGLOBIN,CBC: HEMATOCRIT,CBC: MEAN CORPUSCULAR VOLUME,CBC: MCH,CBC: MCHC,CBC: RDW,CBC: PLATELET COUNT,CBC: ABSOLUTE NEUTROPHILS,CBC: ABSOLUTE LYMPHOCYTES,CBC: NEUTROPHILS,CBC: LYMPHOCYTES,CBC: MONOCYTES,CBC: EOSINOPHILS,CBC: BASOPHILS,METABOLIC: SODIUM,METABOLIC: POTASSIUM,METABOLIC: CHLORIDE,METABOLIC: CARBON DIOXIDE,METABOLIC: ANION GAP,METABOLIC: GLUCOSE,METABOLIC: BUN,METABOLIC: CREATININE,METABOLIC: TOTAL PROTEIN,METABOLIC: ALBUMIN,METABOLIC: CALCIUM,METABOLIC: BILI TOTAL,METABOLIC: AST/SGOT,METABOLIC: ALT/SGPT,METABOLIC: ALK PHOS,URINALYSIS: SPECIFIC GRAVITY,URINALYSIS: PH,URINALYSIS: RED BLOOD CELLS,URINALYSIS: WHITE BLOOD CELLS
361755,DB495784-37B2-4185-8D66-B76E8374099D,5,16 days,M90.631,"Osteitis deformans in neoplastic diseases, rig...",Female,1921-12-14,White,Married,8.5,5,14.7,38,78.5,33.8,28.6,13.6,221.3,76,29.2,5,1.8,0.3,0.3,0.1,138.1,3.8,110.4,25.8,14.2,134.7,24.5,1.1,9.4,5.6,9.2,0.6,16.6,36.8,130,1,6.9,3,3.7
361756,DB495784-37B2-4185-8D66-B76E8374099D,6,2 days,E09.21,Drug or chemical induced diabetes mellitus wit...,Female,1921-12-14,White,Married,8.5,5,14.7,38,78.5,33.8,28.6,13.6,221.3,76,29.2,5,1.8,0.3,0.3,0.1,138.1,3.8,110.4,25.8,14.2,134.7,24.5,1.1,9.4,5.6,9.2,0.6,16.6,36.8,130,1,6.9,3,3.7
361757,B7A09A1C-3648-4E28-A2A2-6B202E3BC39C,1,12 days,C57.22,Malignant neoplasm of left round ligament,Female,1968-10-30,White,Unknown,4.6,3.2,18.1,36.5,75.1,35.3,37,14,359.5,62.2,28.3,8.5,3.8,0.8,0.1,0.1,139,5.4,101,29.1,5.7,96.9,13,1,9,5.5,7,0,28.9,23.2,52,1,5.6,2.9,2.4
361758,B7A09A1C-3648-4E28-A2A2-6B202E3BC39C,2,12 days,C92.12,"Chronic myeloid leukemia, BCR/ABL-positive, in...",Female,1968-10-30,White,Unknown,4.6,3.2,18.1,36.5,75.1,35.3,37,14,359.5,62.2,28.3,8.5,3.8,0.8,0.1,0.1,139,5.4,101,29.1,5.7,96.9,13,1,9,5.5,7,0,28.9,23.2,52,1,5.6,2.9,2.4
361759,B7A09A1C-3648-4E28-A2A2-6B202E3BC39C,3,3 days,D32.0,Benign neoplasm of cerebral meninges,Female,1968-10-30,White,Unknown,4.6,3.2,18.1,36.5,75.1,35.3,37,14,359.5,62.2,28.3,8.5,3.8,0.8,0.1,0.1,139,5.4,101,29.1,5.7,96.9,13,1,9,5.5,7,0,28.9,23.2,52,1,5.6,2.9,2.4


In [76]:
today=dt.today()
today

datetime.date(2020, 9, 2)

In [77]:
data['PatientAge'] = data['PatientDateOfBirth'].apply(lambda x:today.year - x.year -((today.month, today.day) < (x.month, x.day)))
data.head(2)

,PatientID,AdmissionID,AdmissionDuration,PrimaryDiagnosisCode,PrimaryDiagnosisDescription,PatientGender,PatientDateOfBirth,PatientRace,PatientMaritalStatus,CBC: WHITE BLOOD CELL COUNT,CBC: RED BLOOD CELL COUNT,CBC: HEMOGLOBIN,CBC: HEMATOCRIT,CBC: MEAN CORPUSCULAR VOLUME,CBC: MCH,CBC: MCHC,CBC: RDW,CBC: PLATELET COUNT,CBC: ABSOLUTE NEUTROPHILS,CBC: ABSOLUTE LYMPHOCYTES,CBC: NEUTROPHILS,CBC: LYMPHOCYTES,CBC: MONOCYTES,CBC: EOSINOPHILS,CBC: BASOPHILS,METABOLIC: SODIUM,METABOLIC: POTASSIUM,METABOLIC: CHLORIDE,METABOLIC: CARBON DIOXIDE,METABOLIC: ANION GAP,METABOLIC: GLUCOSE,METABOLIC: BUN,METABOLIC: CREATININE,METABOLIC: TOTAL PROTEIN,METABOLIC: ALBUMIN,METABOLIC: CALCIUM,METABOLIC: BILI TOTAL,METABOLIC: AST/SGOT,METABOLIC: ALT/SGPT,METABOLIC: ALK PHOS,URINALYSIS: SPECIFIC GRAVITY,URINALYSIS: PH,URINALYSIS: RED BLOOD CELLS,URINALYSIS: WHITE BLOOD CELLS,PatientAge
0,9380F9E3-1927-42F3-9731-03A74D4E4C6B,5,5 days,M11.07,"Hydroxyapatite deposition disease, ankle and foot",Male,1930-08-12,Unknown,Divorced,9.4,5.1,13.9,35.5,93.6,26.9,35.4,16.3,294.9,71.5,32.3,5.4,3.3,0.4,0.1,0.2,133.5,3.8,92.9,20.1,17.7,114.2,10,1,5.5,3.3,8.6,0,41.8,54.1,62,1,5.7,0.7,5.8,90
1,9380F9E3-1927-42F3-9731-03A74D4E4C6B,1,19 days,D37.030,Neoplasm of uncertain behavior of the parotid ...,Male,1930-08-12,Unknown,Divorced,9.4,5.1,13.9,35.5,93.6,26.9,35.4,16.3,294.9,71.5,32.3,5.4,3.3,0.4,0.1,0.2,133.5,3.8,92.9,20.1,17.7,114.2,10,1,5.5,3.3,8.6,0,41.8,54.1,62,1,5.7,0.7,5.8,90


# 17) Generating the PatientAgeCategory column from  PatientAge

In [78]:
data.loc[(data.PatientAge<18.0),'PatientAgeCategory']='Minor'
data.loc[(data.PatientAge>=18.0) &(data.PatientAge<=35.0),'PatientAgeCategory']='Youth'
data.loc[(data.PatientAge>35.0) &(data.PatientAge<60.0),'PatientAgeCategory']='Adults'
data.loc[(data.PatientAge>=60.0),'PatientAgeCategory']='Senior Citizen'

In [79]:
data.head(5)

,PatientID,AdmissionID,AdmissionDuration,PrimaryDiagnosisCode,PrimaryDiagnosisDescription,PatientGender,PatientDateOfBirth,PatientRace,PatientMaritalStatus,CBC: WHITE BLOOD CELL COUNT,CBC: RED BLOOD CELL COUNT,CBC: HEMOGLOBIN,CBC: HEMATOCRIT,CBC: MEAN CORPUSCULAR VOLUME,CBC: MCH,CBC: MCHC,CBC: RDW,CBC: PLATELET COUNT,CBC: ABSOLUTE NEUTROPHILS,CBC: ABSOLUTE LYMPHOCYTES,CBC: NEUTROPHILS,CBC: LYMPHOCYTES,CBC: MONOCYTES,CBC: EOSINOPHILS,CBC: BASOPHILS,METABOLIC: SODIUM,METABOLIC: POTASSIUM,METABOLIC: CHLORIDE,METABOLIC: CARBON DIOXIDE,METABOLIC: ANION GAP,METABOLIC: GLUCOSE,METABOLIC: BUN,METABOLIC: CREATININE,METABOLIC: TOTAL PROTEIN,METABOLIC: ALBUMIN,METABOLIC: CALCIUM,METABOLIC: BILI TOTAL,METABOLIC: AST/SGOT,METABOLIC: ALT/SGPT,METABOLIC: ALK PHOS,URINALYSIS: SPECIFIC GRAVITY,URINALYSIS: PH,URINALYSIS: RED BLOOD CELLS,URINALYSIS: WHITE BLOOD CELLS,PatientAge,PatientAgeCategory
0,9380F9E3-1927-42F3-9731-03A74D4E4C6B,5,5 days,M11.07,"Hydroxyapatite deposition disease, ankle and foot",Male,1930-08-12,Unknown,Divorced,9.4,5.1,13.9,35.5,93.6,26.9,35.4,16.3,294.9,71.5,32.3,5.4,3.3,0.4,0.1,0.2,133.5,3.8,92.9,20.1,17.7,114.2,10,1,5.5,3.3,8.6,0,41.8,54.1,62,1,5.7,0.7,5.8,90,Senior Citizen
1,9380F9E3-1927-42F3-9731-03A74D4E4C6B,1,19 days,D37.030,Neoplasm of uncertain behavior of the parotid ...,Male,1930-08-12,Unknown,Divorced,9.4,5.1,13.9,35.5,93.6,26.9,35.4,16.3,294.9,71.5,32.3,5.4,3.3,0.4,0.1,0.2,133.5,3.8,92.9,20.1,17.7,114.2,10,1,5.5,3.3,8.6,0,41.8,54.1,62,1,5.7,0.7,5.8,90,Senior Citizen
2,9380F9E3-1927-42F3-9731-03A74D4E4C6B,2,13 days,F31.61,"Bipolar disorder, current episode mixed, mild",Male,1930-08-12,Unknown,Divorced,9.4,5.1,13.9,35.5,93.6,26.9,35.4,16.3,294.9,71.5,32.3,5.4,3.3,0.4,0.1,0.2,133.5,3.8,92.9,20.1,17.7,114.2,10,1,5.5,3.3,8.6,0,41.8,54.1,62,1,5.7,0.7,5.8,90,Senior Citizen
3,9380F9E3-1927-42F3-9731-03A74D4E4C6B,3,13 days,T82.21,Mechanical complication of coronary artery byp...,Male,1930-08-12,Unknown,Divorced,9.4,5.1,13.9,35.5,93.6,26.9,35.4,16.3,294.9,71.5,32.3,5.4,3.3,0.4,0.1,0.2,133.5,3.8,92.9,20.1,17.7,114.2,10,1,5.5,3.3,8.6,0,41.8,54.1,62,1,5.7,0.7,5.8,90,Senior Citizen
4,9380F9E3-1927-42F3-9731-03A74D4E4C6B,4,3 days,C10.2,Malignant neoplasm of lateral wall of oropharynx,Male,1930-08-12,Unknown,Divorced,9.4,5.1,13.9,35.5,93.6,26.9,35.4,16.3,294.9,71.5,32.3,5.4,3.3,0.4,0.1,0.2,133.5,3.8,92.9,20.1,17.7,114.2,10,1,5.5,3.3,8.6,0,41.8,54.1,62,1,5.7,0.7,5.8,90,Senior Citizen


In [80]:
data.drop(columns=['PatientAge'],inplace=True)

In [81]:
Counter(data.PatientAgeCategory)

Counter({'Adults': 135591, 'Senior Citizen': 218978, 'Youth': 7191})

In [82]:
Counter(data.PatientGender)

Counter({'Female': 188096, 'Male': 173664})

In [83]:
Counter(data.PatientRace)

Counter({'African American': 54040,
         'Asian': 83390,
         'Unknown': 46757,
         'White': 177573})

In [84]:
Counter(data.PatientMaritalStatus)

Counter({'Divorced': 39692,
         'Married': 163036,
         'Separated': 18154,
         'Single': 115556,
         'Unknown': 21720,
         'Widowed': 3602})

In [85]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 361760 entries, 0 to 361759
Data columns (total 45 columns):
 #   Column                         Non-Null Count   Dtype          
---  ------                         --------------   -----          
 0   PatientID                      361760 non-null  object         
 1   AdmissionID                    361760 non-null  int64          
 2   AdmissionDuration              361760 non-null  timedelta64[ns]
 3   PrimaryDiagnosisCode           361760 non-null  object         
 4   PrimaryDiagnosisDescription    361760 non-null  object         
 5   PatientGender                  361760 non-null  object         
 6   PatientDateOfBirth             361760 non-null  object         
 7   PatientRace                    361760 non-null  object         
 8   PatientMaritalStatus           361760 non-null  object         
 9   CBC: WHITE BLOOD CELL COUNT    361746 non-null  object         
 10  CBC: RED BLOOD CELL COUNT      361750 non-null  object  

In [86]:
data.shape

(361760, 45)

# 18) Handle the null values present in the Lab group

In [87]:
data.isna().sum().to_frame().T

,PatientID,AdmissionID,AdmissionDuration,PrimaryDiagnosisCode,PrimaryDiagnosisDescription,PatientGender,PatientDateOfBirth,PatientRace,PatientMaritalStatus,CBC: WHITE BLOOD CELL COUNT,CBC: RED BLOOD CELL COUNT,CBC: HEMOGLOBIN,CBC: HEMATOCRIT,CBC: MEAN CORPUSCULAR VOLUME,CBC: MCH,CBC: MCHC,CBC: RDW,CBC: PLATELET COUNT,CBC: ABSOLUTE NEUTROPHILS,CBC: ABSOLUTE LYMPHOCYTES,CBC: NEUTROPHILS,CBC: LYMPHOCYTES,CBC: MONOCYTES,CBC: EOSINOPHILS,CBC: BASOPHILS,METABOLIC: SODIUM,METABOLIC: POTASSIUM,METABOLIC: CHLORIDE,METABOLIC: CARBON DIOXIDE,METABOLIC: ANION GAP,METABOLIC: GLUCOSE,METABOLIC: BUN,METABOLIC: CREATININE,METABOLIC: TOTAL PROTEIN,METABOLIC: ALBUMIN,METABOLIC: CALCIUM,METABOLIC: BILI TOTAL,METABOLIC: AST/SGOT,METABOLIC: ALT/SGPT,METABOLIC: ALK PHOS,URINALYSIS: SPECIFIC GRAVITY,URINALYSIS: PH,URINALYSIS: RED BLOOD CELLS,URINALYSIS: WHITE BLOOD CELLS,PatientAgeCategory
0,0,0,0,0,0,0,0,0,0,14,10,11,3,11,11,6,6,10,4,6,6,4,6,9,7,9,11,8,10,5,12,8,10,7,10,9,8,7,9,3,11,6,7,3,0


In [88]:
type(data)

pandas.core.frame.DataFrame

In [89]:
data.fillna(method='ffill',inplace=True)

In [90]:
data.isna().sum().to_frame().T

,PatientID,AdmissionID,AdmissionDuration,PrimaryDiagnosisCode,PrimaryDiagnosisDescription,PatientGender,PatientDateOfBirth,PatientRace,PatientMaritalStatus,CBC: WHITE BLOOD CELL COUNT,CBC: RED BLOOD CELL COUNT,CBC: HEMOGLOBIN,CBC: HEMATOCRIT,CBC: MEAN CORPUSCULAR VOLUME,CBC: MCH,CBC: MCHC,CBC: RDW,CBC: PLATELET COUNT,CBC: ABSOLUTE NEUTROPHILS,CBC: ABSOLUTE LYMPHOCYTES,CBC: NEUTROPHILS,CBC: LYMPHOCYTES,CBC: MONOCYTES,CBC: EOSINOPHILS,CBC: BASOPHILS,METABOLIC: SODIUM,METABOLIC: POTASSIUM,METABOLIC: CHLORIDE,METABOLIC: CARBON DIOXIDE,METABOLIC: ANION GAP,METABOLIC: GLUCOSE,METABOLIC: BUN,METABOLIC: CREATININE,METABOLIC: TOTAL PROTEIN,METABOLIC: ALBUMIN,METABOLIC: CALCIUM,METABOLIC: BILI TOTAL,METABOLIC: AST/SGOT,METABOLIC: ALT/SGPT,METABOLIC: ALK PHOS,URINALYSIS: SPECIFIC GRAVITY,URINALYSIS: PH,URINALYSIS: RED BLOOD CELLS,URINALYSIS: WHITE BLOOD CELLS,PatientAgeCategory
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [91]:
data.shape

(361760, 45)

In [92]:
data=pd.DataFrame(data.values.reshape(361760,45),columns=['PatientID','AdmissionID','AdmissionDuration',
                                                          'PrimaryDiagnosisCode','PrimaryDiagnosisDescription','PatientGender',
                                                          'PatientDateOfBirth','PatientMaritalStatus','PatientRace','PatientAgeCategory',
   'CBC: WHITE BLOOD CELL COUNT','CBC: RED BLOOD CELL COUNT','CBC: HEMOGLOBIN','CBC: HEMATOCRIT',
   'CBC: MEAN CORPUSCULAR VOLUME','CBC: MCH','CBC: MCHC','CBC: RDW',
   'CBC: PLATELET COUNT','CBC: ABSOLUTE NEUTROPHILS',
   'CBC: ABSOLUTE LYMPHOCYTES','CBC: NEUTROPHILS','CBC: LYMPHOCYTES',
   'CBC: MONOCYTES','CBC: EOSINOPHILS','CBC: BASOPHILS',
   'METABOLIC: SODIUM','METABOLIC: POTASSIUM', 'METABOLIC: CHLORIDE',
   'METABOLIC: CARBON DIOXIDE','METABOLIC: ANION GAP',
   'METABOLIC: GLUCOSE','METABOLIC: BUN','METABOLIC: CREATININE',
   'METABOLIC: TOTAL PROTEIN','METABOLIC: ALBUMIN','METABOLIC: CALCIUM',
   'METABOLIC: BILI TOTAL','METABOLIC: AST/SGOT','METABOLIC: ALT/SGPT',
   'METABOLIC: ALK PHOS','URINALYSIS: SPECIFIC GRAVITY','URINALYSIS: PH',
   'URINALYSIS: RED BLOOD CELLS','URINALYSIS: WHITE BLOOD CELLS'])

In [93]:
data.shape

(361760, 45)

In [94]:
data.isna().sum().to_frame().T

,PatientID,AdmissionID,AdmissionDuration,PrimaryDiagnosisCode,PrimaryDiagnosisDescription,PatientGender,PatientDateOfBirth,PatientMaritalStatus,PatientRace,PatientAgeCategory,CBC: WHITE BLOOD CELL COUNT,CBC: RED BLOOD CELL COUNT,CBC: HEMOGLOBIN,CBC: HEMATOCRIT,CBC: MEAN CORPUSCULAR VOLUME,CBC: MCH,CBC: MCHC,CBC: RDW,CBC: PLATELET COUNT,CBC: ABSOLUTE NEUTROPHILS,CBC: ABSOLUTE LYMPHOCYTES,CBC: NEUTROPHILS,CBC: LYMPHOCYTES,CBC: MONOCYTES,CBC: EOSINOPHILS,CBC: BASOPHILS,METABOLIC: SODIUM,METABOLIC: POTASSIUM,METABOLIC: CHLORIDE,METABOLIC: CARBON DIOXIDE,METABOLIC: ANION GAP,METABOLIC: GLUCOSE,METABOLIC: BUN,METABOLIC: CREATININE,METABOLIC: TOTAL PROTEIN,METABOLIC: ALBUMIN,METABOLIC: CALCIUM,METABOLIC: BILI TOTAL,METABOLIC: AST/SGOT,METABOLIC: ALT/SGPT,METABOLIC: ALK PHOS,URINALYSIS: SPECIFIC GRAVITY,URINALYSIS: PH,URINALYSIS: RED BLOOD CELLS,URINALYSIS: WHITE BLOOD CELLS
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [95]:
data.to_csv('/content/drive/My Drive/Colab Notebooks/EHR_TransformedData.csv')

# 19) Using Label Encoding to convert categorical data into numericals

In [ ]:
from sklearn.preprocessing import LabelEncoder
enc=LabelEncoder()

data['AdmissionDuration']=enc.fit_transform(data['AdmissionDuration']).astype('int16')
data['PatientGender']=enc.fit_transform(data['PatientGender']).astype('int16')
data['PatientMaritalStatus']=enc.fit_transform(data['PatientMaritalStatus']).astype('int16')
data['PatientRace']=enc.fit_transform(data['PatientRace']).astype('int16')
data['PatientAgeCategory']=enc.fit_transform(data['PatientAgeCategory']).astype('int16')


data['CBC: WHITE BLOOD CELL COUNT']=enc.fit_transform(data['CBC: WHITE BLOOD CELL COUNT']).astype('int16')
data['CBC: RED BLOOD CELL COUNT']=enc.fit_transform(data['CBC: RED BLOOD CELL COUNT']).astype('int16')
data['CBC: HEMOGLOBIN']=enc.fit_transform(data['CBC: HEMOGLOBIN']).astype('int16')
data['CBC: HEMATOCRIT']=enc.fit_transform(data['CBC: HEMATOCRIT']).astype('int16')
data['CBC: MEAN CORPUSCULAR VOLUME']=enc.fit_transform(data['CBC: MEAN CORPUSCULAR VOLUME']).astype('int16')
data['CBC: MCH']=enc.fit_transform(data['CBC: MCH']).astype('int16')
data['CBC: MCHC']=enc.fit_transform(data['CBC: MCHC']).astype('int16')
data['CBC: RDW']=enc.fit_transform(data['CBC: RDW']).astype('int16')
data['CBC: PLATELET COUNT']=enc.fit_transform(data['CBC: PLATELET COUNT']).astype('int16')
data['CBC: ABSOLUTE NEUTROPHILS']=enc.fit_transform(data['CBC: ABSOLUTE NEUTROPHILS']).astype('int16')
data['CBC: ABSOLUTE LYMPHOCYTES']=enc.fit_transform(data['CBC: ABSOLUTE LYMPHOCYTES']).astype('int16')
data['CBC: LYMPHOCYTES']=enc.fit_transform(data['CBC: LYMPHOCYTES']).astype('int16')
data['CBC: NEUTROPHILS']=enc.fit_transform(data['CBC: NEUTROPHILS']).astype('int16')
data['CBC: MONOCYTES']=enc.fit_transform(data['CBC: MONOCYTES']).astype('int16')
data['CBC: EOSINOPHILS']=enc.fit_transform(data['CBC: EOSINOPHILS']).astype('int16')
data['CBC: BASOPHILS']=enc.fit_transform(data['CBC: BASOPHILS']).astype('int16')
data['METABOLIC: SODIUM']=enc.fit_transform(data['METABOLIC: SODIUM']).astype('int16')
data['METABOLIC: POTASSIUM']=enc.fit_transform(data['METABOLIC: POTASSIUM']).astype('int16')
data['METABOLIC: CHLORIDE']=enc.fit_transform(data['METABOLIC: CHLORIDE']).astype('int16')
data['METABOLIC: CARBON DIOXIDE']=enc.fit_transform(data['METABOLIC: CARBON DIOXIDE']).astype('int16')
data['METABOLIC: ANION GAP']=enc.fit_transform(data['METABOLIC: ANION GAP']).astype('int16')
data['METABOLIC: GLUCOSE']=enc.fit_transform(data['METABOLIC: GLUCOSE']).astype('int16')
data['METABOLIC: BUN']=enc.fit_transform(data['METABOLIC: BUN']).astype('int16')
data['METABOLIC: CREATININE']=enc.fit_transform(data['METABOLIC: CREATININE']).astype('int16')
data['METABOLIC: TOTAL PROTEIN']=enc.fit_transform(data['METABOLIC: TOTAL PROTEIN']).astype('int16')
data['METABOLIC: ALBUMIN']=enc.fit_transform(data['METABOLIC: ALBUMIN']).astype('int16')
data['METABOLIC: CALCIUM']=enc.fit_transform(data['METABOLIC: CALCIUM']).astype('int16')
data['METABOLIC: BILI TOTAL']=enc.fit_transform(data['METABOLIC: BILI TOTAL']).astype('int16')
data['METABOLIC: AST/SGOT']=enc.fit_transform(data['METABOLIC: AST/SGOT']).astype('int16')
data['METABOLIC: ALT/SGPT']=enc.fit_transform(data['METABOLIC: ALT/SGPT']).astype('int16')
data['METABOLIC: ALK PHOS']=enc.fit_transform(data['METABOLIC: ALK PHOS']).astype('int16')
data['URINALYSIS: SPECIFIC GRAVITY']=enc.fit_transform(data['URINALYSIS: SPECIFIC GRAVITY']).astype('int16')
data['URINALYSIS: PH']=enc.fit_transform(data['URINALYSIS: PH']).astype('int16')
data['URINALYSIS: RED BLOOD CELLS']=enc.fit_transform(data['URINALYSIS: RED BLOOD CELLS']).astype('int16')
data['URINALYSIS: WHITE BLOOD CELLS']=enc.fit_transform(data['URINALYSIS: WHITE BLOOD CELLS']).astype('int16')

In [ ]:
data.info()

# 20) Finding correlations between different columns in Lab group and generating correlation matrix

In [ ]:
corr=data.corr()
corr

In [ ]:
plt.figure(figsize=(25,25))
sns.heatmap(corr,cmap='inferno',vmax=.3,center=0,square=True,linewidths=.6,cbar_kws={'shrink':.5},annot=True)

# 21) Processing dataframes with billions of rows

In [ ]:
# !pip3 install --upgrade pip

In [ ]:
# !pip3 -V

In [ ]:
!pip3 install --pre vaex

In [ ]:
!pip3 install pyarrow

In [ ]:
!pip3 install --upgrade pyarrow

Requirement already up-to-date: pyarrow in /usr/local/lib/python3.6/dist-packages (1.0.1)


In [ ]:
!pip3 install vaex --no-cache-dir 

In [ ]:
!pip3 install --upgrade pip

Requirement already up-to-date: pip in /usr/local/lib/python3.6/dist-packages (20.2.2)


In [ ]:
!pip3 install -U setuptools

Requirement already up-to-date: setuptools in /usr/local/lib/python3.6/dist-packages (50.0.0)


In [ ]:
import vaex
import vaex.ml

In [ ]:
data.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 361760 entries, 0 to 361759
Data columns (total 45 columns):
 #   Column                         Non-Null Count   Dtype 
---  ------                         --------------   ----- 
 0   PatientID                      361760 non-null  object
 1   AdmissionID                    361760 non-null  object
 2   AdmissionDuration              361760 non-null  int16 
 3   PrimaryDiagnosisCode           361760 non-null  object
 4   PrimaryDiagnosisDescription    361760 non-null  object
 5   PatientGender                  361760 non-null  int16 
 6   PatientDateOfBirth             361760 non-null  object
 7   PatientMaritalStatus           361760 non-null  int16 
 8   PatientRace                    361760 non-null  int16 
 9   PatientAgeCategory             361760 non-null  int16 
 10  CBC: WHITE BLOOD CELL COUNT    361760 non-null  int16 
 11  CBC: RED BLOOD CELL COUNT      361760 non-null  int16 
 12  CBC: HEMOGLOBIN                361760 non-nu

In [ ]:
file_path='EHR_TransformedData.csv'
data.to_csv(file_path,index=False)

In [ ]:
!pip3 install --upgrade ipython

Requirement already up-to-date: ipython in /usr/local/lib/python3.6/dist-packages (7.16.1)


In [ ]:
!pip3 show ipython

Name: ipython
Version: 7.16.1
Summary: IPython: Productive Interactive Computing
Home-page: https://ipython.org
Author: The IPython Development Team
Author-email: ipython-dev@python.org
License: BSD
Location: /usr/local/lib/python3.6/dist-packages
Requires: pickleshare, pygments, jedi, prompt-toolkit, traitlets, backcall, setuptools, decorator, pexpect
Required-by: lucid, jupyter-console, ipywidgets, ipython-sql, ipykernel, google-colab, cufflinks


In [ ]:
data_vaex=vaex.from_csv(file_path,convert=True,chunk_size=500)
data_vaex

#,PatientID,AdmissionID,AdmissionDuration,PrimaryDiagnosisCode,PrimaryDiagnosisDescription,PatientGender,PatientDateOfBirth,PatientMaritalStatus,PatientRace,PatientAgeCategory,CBC: WHITE BLOOD CELL COUNT,CBC: RED BLOOD CELL COUNT,CBC: HEMOGLOBIN,CBC: HEMATOCRIT,CBC: MEAN CORPUSCULAR VOLUME,CBC: MCH,CBC: MCHC,CBC: RDW,CBC: PLATELET COUNT,CBC: ABSOLUTE NEUTROPHILS,CBC: ABSOLUTE LYMPHOCYTES,CBC: NEUTROPHILS,CBC: LYMPHOCYTES,CBC: MONOCYTES,CBC: EOSINOPHILS,CBC: BASOPHILS,METABOLIC: SODIUM,METABOLIC: POTASSIUM,METABOLIC: CHLORIDE,METABOLIC: CARBON DIOXIDE,METABOLIC: ANION GAP,METABOLIC: GLUCOSE,METABOLIC: BUN,METABOLIC: CREATININE,METABOLIC: TOTAL PROTEIN,METABOLIC: ALBUMIN,METABOLIC: CALCIUM,METABOLIC: BILI TOTAL,METABOLIC: AST/SGOT,METABOLIC: ALT/SGPT,METABOLIC: ALK PHOS,URINALYSIS: SPECIFIC GRAVITY,URINALYSIS: PH,URINALYSIS: RED BLOOD CELLS,URINALYSIS: WHITE BLOOD CELLS
0,9380F9E3-1927-42F3-9731-03A74D4E4C6B,5,3,M11.07,"Hydroxyapatite deposition disease, ankle and foot",1,1930-08-12,2,0,85,21,39,55,237,49,74,63,1749,115,173,55,28,3,0,2,85,8,180,21,77,142,0,5,6,8,37,0,298,391,721,0,12,7,58,1
1,9380F9E3-1927-42F3-9731-03A74D4E4C6B,1,17,D37.030,'Neoplasm of uncertain behavior of the parotid s...,1,1930-08-12,2,0,85,21,39,55,237,49,74,63,1749,115,173,55,28,3,0,2,85,8,180,21,77,142,0,5,6,8,37,0,298,391,721,0,12,7,58,1
2,9380F9E3-1927-42F3-9731-03A74D4E4C6B,2,11,F31.61,"Bipolar disorder, current episode mixed, mild",1,1930-08-12,2,0,85,21,39,55,237,49,74,63,1749,115,173,55,28,3,0,2,85,8,180,21,77,142,0,5,6,8,37,0,298,391,721,0,12,7,58,1
3,9380F9E3-1927-42F3-9731-03A74D4E4C6B,3,11,T82.21,'Mechanical complication of coronary artery bypa...,1,1930-08-12,2,0,85,21,39,55,237,49,74,63,1749,115,173,55,28,3,0,2,85,8,180,21,77,142,0,5,6,8,37,0,298,391,721,0,12,7,58,1
4,9380F9E3-1927-42F3-9731-03A74D4E4C6B,4,1,C10.2,Malignant neoplasm of lateral wall of oropharynx,1,1930-08-12,2,0,85,21,39,55,237,49,74,63,1749,115,173,55,28,3,0,2,85,8,180,21,77,142,0,5,6,8,37,0,298,391,721,0,12,7,58,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"361,755",DB495784-37B2-4185-8D66-B76E8374099D,5,14,M90.631,"'Osteitis deformans in neoplastic diseases, righ...",0,1921-12-14,3,1,76,20,47,80,86,118,6,36,1013,160,142,51,13,2,2,1,131,8,104,78,42,347,145,6,45,31,43,6,46,218,300,0,24,30,37,1
"361,756",DB495784-37B2-4185-8D66-B76E8374099D,6,0,E09.21,'Drug or chemical induced diabetes mellitus with...,0,1921-12-14,3,1,76,20,47,80,86,118,6,36,1013,160,142,51,13,2,2,1,131,8,104,78,42,347,145,6,45,31,43,6,46,218,300,0,24,30,37,1
"361,757",B7A09A1C-3648-4E28-A2A2-6B202E3BC39C,1,10,C57.22,Malignant neoplasm of left round ligament,0,1968-10-30,3,4,37,2,81,65,52,133,90,40,2395,22,133,86,33,7,0,1,140,24,10,111,108,770,30,5,41,30,21,0,169,82,621,0,11,29,24,0
"361,758",B7A09A1C-3648-4E28-A2A2-6B202E3BC39C,2,10,C92.12,"'Chronic myeloid leukemia, BCR/ABL-positive, in ...",0,1968-10-30,3,4,37,2,81,65,52,133,90,40,2395,22,133,86,33,7,0,1,140,24,10,111,108,770,30,5,41,30,21,0,169,82,621,0,11,29,24,0


In [ ]:
type(data_vaex)

vaex.hdf5.dataset.Hdf5MemoryMapped

In [ ]:
data_vaex['PrimaryDiagnosisCode']

Expression = PrimaryDiagnosisCode
Length: 361,760 dtype: str (column)
-----------------------------------
     0   M11.07
     1  D37.030
     2   F31.61
     3   T82.21
     4    C10.2
      ...      
361755  M90.631
361756   E09.21
361757   C57.22
361758   C92.12
361759    D32.0

In [ ]:
data_vaex=vaex.open('EHR_TransformedData.csv.hdf5')
data_vaex

#,PatientID,AdmissionID,AdmissionDuration,PrimaryDiagnosisCode,PrimaryDiagnosisDescription,PatientGender,PatientDateOfBirth,PatientMaritalStatus,PatientRace,PatientAgeCategory,CBC: WHITE BLOOD CELL COUNT,CBC: RED BLOOD CELL COUNT,CBC: HEMOGLOBIN,CBC: HEMATOCRIT,CBC: MEAN CORPUSCULAR VOLUME,CBC: MCH,CBC: MCHC,CBC: RDW,CBC: PLATELET COUNT,CBC: ABSOLUTE NEUTROPHILS,CBC: ABSOLUTE LYMPHOCYTES,CBC: NEUTROPHILS,CBC: LYMPHOCYTES,CBC: MONOCYTES,CBC: EOSINOPHILS,CBC: BASOPHILS,METABOLIC: SODIUM,METABOLIC: POTASSIUM,METABOLIC: CHLORIDE,METABOLIC: CARBON DIOXIDE,METABOLIC: ANION GAP,METABOLIC: GLUCOSE,METABOLIC: BUN,METABOLIC: CREATININE,METABOLIC: TOTAL PROTEIN,METABOLIC: ALBUMIN,METABOLIC: CALCIUM,METABOLIC: BILI TOTAL,METABOLIC: AST/SGOT,METABOLIC: ALT/SGPT,METABOLIC: ALK PHOS,URINALYSIS: SPECIFIC GRAVITY,URINALYSIS: PH,URINALYSIS: RED BLOOD CELLS,URINALYSIS: WHITE BLOOD CELLS
0,9380F9E3-1927-42F3-9731-03A74D4E4C6B,5,3,M11.07,"Hydroxyapatite deposition disease, ankle and foot",1,1930-08-12,2,0,85,21,39,55,237,49,74,63,1749,115,173,55,28,3,0,2,85,8,180,21,77,142,0,5,6,8,37,0,298,391,721,0,12,7,58,1
1,9380F9E3-1927-42F3-9731-03A74D4E4C6B,1,17,D37.030,'Neoplasm of uncertain behavior of the parotid s...,1,1930-08-12,2,0,85,21,39,55,237,49,74,63,1749,115,173,55,28,3,0,2,85,8,180,21,77,142,0,5,6,8,37,0,298,391,721,0,12,7,58,1
2,9380F9E3-1927-42F3-9731-03A74D4E4C6B,2,11,F31.61,"Bipolar disorder, current episode mixed, mild",1,1930-08-12,2,0,85,21,39,55,237,49,74,63,1749,115,173,55,28,3,0,2,85,8,180,21,77,142,0,5,6,8,37,0,298,391,721,0,12,7,58,1
3,9380F9E3-1927-42F3-9731-03A74D4E4C6B,3,11,T82.21,'Mechanical complication of coronary artery bypa...,1,1930-08-12,2,0,85,21,39,55,237,49,74,63,1749,115,173,55,28,3,0,2,85,8,180,21,77,142,0,5,6,8,37,0,298,391,721,0,12,7,58,1
4,9380F9E3-1927-42F3-9731-03A74D4E4C6B,4,1,C10.2,Malignant neoplasm of lateral wall of oropharynx,1,1930-08-12,2,0,85,21,39,55,237,49,74,63,1749,115,173,55,28,3,0,2,85,8,180,21,77,142,0,5,6,8,37,0,298,391,721,0,12,7,58,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"361,755",DB495784-37B2-4185-8D66-B76E8374099D,5,14,M90.631,"'Osteitis deformans in neoplastic diseases, righ...",0,1921-12-14,3,1,76,20,47,80,86,118,6,36,1013,160,142,51,13,2,2,1,131,8,104,78,42,347,145,6,45,31,43,6,46,218,300,0,24,30,37,1
"361,756",DB495784-37B2-4185-8D66-B76E8374099D,6,0,E09.21,'Drug or chemical induced diabetes mellitus with...,0,1921-12-14,3,1,76,20,47,80,86,118,6,36,1013,160,142,51,13,2,2,1,131,8,104,78,42,347,145,6,45,31,43,6,46,218,300,0,24,30,37,1
"361,757",B7A09A1C-3648-4E28-A2A2-6B202E3BC39C,1,10,C57.22,Malignant neoplasm of left round ligament,0,1968-10-30,3,4,37,2,81,65,52,133,90,40,2395,22,133,86,33,7,0,1,140,24,10,111,108,770,30,5,41,30,21,0,169,82,621,0,11,29,24,0
"361,758",B7A09A1C-3648-4E28-A2A2-6B202E3BC39C,2,10,C92.12,"'Chronic myeloid leukemia, BCR/ABL-positive, in ...",0,1968-10-30,3,4,37,2,81,65,52,133,90,40,2395,22,133,86,33,7,0,1,140,24,10,111,108,770,30,5,41,30,21,0,169,82,621,0,11,29,24,0


In [ ]:
type(data_vaex)

vaex.hdf5.dataset.Hdf5MemoryMapped

In [ ]:
data_vaex.dtypes

PatientID                        <class 'str'>
AdmissionID                              int64
AdmissionDuration                        int64
PrimaryDiagnosisCode             <class 'str'>
PrimaryDiagnosisDescription      <class 'str'>
PatientGender                            int64
PatientDateOfBirth               <class 'str'>
PatientMaritalStatus                     int64
PatientRace                              int64
PatientAgeCategory                       int64
CBC: WHITE BLOOD CELL COUNT              int64
CBC: RED BLOOD CELL COUNT                int64
CBC: HEMOGLOBIN                          int64
CBC: HEMATOCRIT                          int64
CBC: MEAN CORPUSCULAR VOLUME             int64
CBC: MCH                                 int64
CBC: MCHC                                int64
CBC: RDW                                 int64
CBC: PLATELET COUNT                      int64
CBC: ABSOLUTE NEUTROPHILS                int64
CBC: ABSOLUTE LYMPHOCYTES                int64
CBC: NEUTROPH

# 22) Define features variable

In [ ]:
features=['PatientGender','PatientMaritalStatus','PatientRace','PatientAgeCategory',
  'CBC: WHITE BLOOD CELL COUNT','CBC: RED BLOOD CELL COUNT', 'CBC: HEMOGLOBIN', 'CBC: HEMATOCRIT',
   'CBC: MEAN CORPUSCULAR VOLUME', 'CBC: MCH', 'CBC: MCHC', 'CBC: RDW',
   'CBC: PLATELET COUNT', 'CBC: ABSOLUTE NEUTROPHILS',
   'CBC: ABSOLUTE LYMPHOCYTES', 'CBC: NEUTROPHILS', 'CBC: LYMPHOCYTES',
   'CBC: MONOCYTES', 'CBC: EOSINOPHILS', 'CBC: BASOPHILS',
   'METABOLIC: SODIUM', 'METABOLIC: POTASSIUM', 'METABOLIC: CHLORIDE',
   'METABOLIC: CARBON DIOXIDE', 'METABOLIC: ANION GAP',
   'METABOLIC: GLUCOSE', 'METABOLIC: BUN', 'METABOLIC: CREATININE',
   'METABOLIC: TOTAL PROTEIN', 'METABOLIC: ALBUMIN', 'METABOLIC: CALCIUM',
   'METABOLIC: BILI TOTAL', 'METABOLIC: AST/SGOT', 'METABOLIC: ALT/SGPT',
   'METABOLIC: ALK PHOS', 'URINALYSIS: SPECIFIC GRAVITY', 'URINALYSIS: PH',
   'URINALYSIS: RED BLOOD CELLS', 'URINALYSIS: WHITE BLOOD CELLS']

In [ ]:
type(features)

list

In [ ]:
target='PrimaryDiagnosisCode'
target

'PrimaryDiagnosisCode'

In [ ]:
type(target)

str

# 23) Split the data into test and train

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
data_vaex_train, data_vaex_test = data_vaex.ml.train_test_split(test_size=0.3, verbose=False)
print(" data train shape = ",data_vaex_train.shape)
print(" data test shape = ",data_vaex_test.shape)

 data train shape =  (253232, 45)
 data test shape =  (108528, 45)


In [ ]:
data_vaex_train.head()

#,PatientID,AdmissionID,AdmissionDuration,PrimaryDiagnosisCode,PrimaryDiagnosisDescription,PatientGender,PatientDateOfBirth,PatientMaritalStatus,PatientRace,PatientAgeCategory,CBC: WHITE BLOOD CELL COUNT,CBC: RED BLOOD CELL COUNT,CBC: HEMOGLOBIN,CBC: HEMATOCRIT,CBC: MEAN CORPUSCULAR VOLUME,CBC: MCH,CBC: MCHC,CBC: RDW,CBC: PLATELET COUNT,CBC: ABSOLUTE NEUTROPHILS,CBC: ABSOLUTE LYMPHOCYTES,CBC: NEUTROPHILS,CBC: LYMPHOCYTES,CBC: MONOCYTES,CBC: EOSINOPHILS,CBC: BASOPHILS,METABOLIC: SODIUM,METABOLIC: POTASSIUM,METABOLIC: CHLORIDE,METABOLIC: CARBON DIOXIDE,METABOLIC: ANION GAP,METABOLIC: GLUCOSE,METABOLIC: BUN,METABOLIC: CREATININE,METABOLIC: TOTAL PROTEIN,METABOLIC: ALBUMIN,METABOLIC: CALCIUM,METABOLIC: BILI TOTAL,METABOLIC: AST/SGOT,METABOLIC: ALT/SGPT,METABOLIC: ALK PHOS,URINALYSIS: SPECIFIC GRAVITY,URINALYSIS: PH,URINALYSIS: RED BLOOD CELLS,URINALYSIS: WHITE BLOOD CELLS
0,A460A2E5-9163-4B4A-86B5-D47B7A0B20E7,3,8,D20.1,Benign neoplasm of soft tissue of peritoneum,1,1965-08-07,1,3,69,23,9,83,195,79,38,9,638,79,112,16,16,4,2,0,115,14,242,111,3,649,174,5,36,19,41,3,98,448,895,0,17,32,24,0
1,A460A2E5-9163-4B4A-86B5-D47B7A0B20E7,4,17,C76.42,Malignant neoplasm of left upper limb,1,1965-08-07,1,3,69,23,9,83,195,79,38,9,638,79,112,16,16,4,2,0,115,14,242,111,3,649,174,5,36,19,41,3,98,448,895,0,17,32,24,0
2,A460A2E5-9163-4B4A-86B5-D47B7A0B20E7,5,14,C48.8,'Malignant neoplasm of overlapping sites of retr...,1,1965-08-07,1,3,69,23,9,83,195,79,38,9,638,79,112,16,16,4,2,0,115,14,242,111,3,649,174,5,36,19,41,3,98,448,895,0,17,32,24,0
3,2CB0A710-EF7B-4549-8371-338BC0F7C031,1,5,M90.522,"'Osteonecrosis in diseases classified elsewhere,...",1,1958-07-01,3,3,22,18,77,115,198,49,26,77,2821,89,37,86,6,1,4,1,30,29,155,173,86,681,204,4,28,20,40,4,37,471,723,0,24,3,51,1
4,2CB0A710-EF7B-4549-8371-338BC0F7C031,2,2,C60.0,Malignant neoplasm of prepuce,1,1958-07-01,3,3,22,18,77,115,198,49,26,77,2821,89,37,86,6,1,4,1,30,29,155,173,86,681,204,4,28,20,40,4,37,471,723,0,24,3,51,1
5,2CB0A710-EF7B-4549-8371-338BC0F7C031,3,14,E08.36,'Diabetes mellitus due to underlying condition w...,1,1958-07-01,3,3,22,18,77,115,198,49,26,77,2821,89,37,86,6,1,4,1,30,29,155,173,86,681,204,4,28,20,40,4,37,471,723,0,24,3,51,1
6,2CB0A710-EF7B-4549-8371-338BC0F7C031,4,9,C17,Malignant neoplasm of small intestine,1,1958-07-01,3,3,22,18,77,115,198,49,26,77,2821,89,37,86,6,1,4,1,30,29,155,173,86,681,204,4,28,20,40,4,37,471,723,0,24,3,51,1
7,6A369DF8-31DD-4573-851F-4B5517D7F92C,1,17,C20,Malignant neoplasm of rectum,0,1942-08-31,1,0,4,24,87,9,35,161,106,1,1250,151,9,62,24,7,3,2,172,1,157,47,27,271,159,1,13,15,23,9,205,376,179,0,24,24,57,1
8,6A369DF8-31DD-4573-851F-4B5517D7F92C,2,11,M24.212,"Disorder of ligament, left shoulder",0,1942-08-31,1,0,4,24,87,9,35,161,106,1,1250,151,9,62,24,7,3,2,172,1,157,47,27,271,159,1,13,15,23,9,205,376,179,0,24,24,57,1
9,7EAE4204-28D4-4DF0-A8B8-85B43A2E91AA,1,10,C40.3,Malignant neoplasm of short bones of lower limb,0,1985-07-19,3,1,89,13,31,134,169,93,54,29,388,23,147,9,7,7,2,1,167,11,244,102,22,211,99,3,11,26,24,3,281,69,6,0,10,33,7,2


In [ ]:
data_vaex_test.head()

#,PatientID,AdmissionID,AdmissionDuration,PrimaryDiagnosisCode,PrimaryDiagnosisDescription,PatientGender,PatientDateOfBirth,PatientMaritalStatus,PatientRace,PatientAgeCategory,CBC: WHITE BLOOD CELL COUNT,CBC: RED BLOOD CELL COUNT,CBC: HEMOGLOBIN,CBC: HEMATOCRIT,CBC: MEAN CORPUSCULAR VOLUME,CBC: MCH,CBC: MCHC,CBC: RDW,CBC: PLATELET COUNT,CBC: ABSOLUTE NEUTROPHILS,CBC: ABSOLUTE LYMPHOCYTES,CBC: NEUTROPHILS,CBC: LYMPHOCYTES,CBC: MONOCYTES,CBC: EOSINOPHILS,CBC: BASOPHILS,METABOLIC: SODIUM,METABOLIC: POTASSIUM,METABOLIC: CHLORIDE,METABOLIC: CARBON DIOXIDE,METABOLIC: ANION GAP,METABOLIC: GLUCOSE,METABOLIC: BUN,METABOLIC: CREATININE,METABOLIC: TOTAL PROTEIN,METABOLIC: ALBUMIN,METABOLIC: CALCIUM,METABOLIC: BILI TOTAL,METABOLIC: AST/SGOT,METABOLIC: ALT/SGPT,METABOLIC: ALK PHOS,URINALYSIS: SPECIFIC GRAVITY,URINALYSIS: PH,URINALYSIS: RED BLOOD CELLS,URINALYSIS: WHITE BLOOD CELLS
0,9380F9E3-1927-42F3-9731-03A74D4E4C6B,5,3,M11.07,"Hydroxyapatite deposition disease, ankle and foot",1,1930-08-12,2,0,85,21,39,55,237,49,74,63,1749,115,173,55,28,3,0,2,85,8,180,21,77,142,0,5,6,8,37,0,298,391,721,0,12,7,58,1
1,9380F9E3-1927-42F3-9731-03A74D4E4C6B,1,17,D37.030,'Neoplasm of uncertain behavior of the parotid s...,1,1930-08-12,2,0,85,21,39,55,237,49,74,63,1749,115,173,55,28,3,0,2,85,8,180,21,77,142,0,5,6,8,37,0,298,391,721,0,12,7,58,1
2,9380F9E3-1927-42F3-9731-03A74D4E4C6B,2,11,F31.61,"Bipolar disorder, current episode mixed, mild",1,1930-08-12,2,0,85,21,39,55,237,49,74,63,1749,115,173,55,28,3,0,2,85,8,180,21,77,142,0,5,6,8,37,0,298,391,721,0,12,7,58,1
3,9380F9E3-1927-42F3-9731-03A74D4E4C6B,3,11,T82.21,'Mechanical complication of coronary artery bypa...,1,1930-08-12,2,0,85,21,39,55,237,49,74,63,1749,115,173,55,28,3,0,2,85,8,180,21,77,142,0,5,6,8,37,0,298,391,721,0,12,7,58,1
4,9380F9E3-1927-42F3-9731-03A74D4E4C6B,4,1,C10.2,Malignant neoplasm of lateral wall of oropharynx,1,1930-08-12,2,0,85,21,39,55,237,49,74,63,1749,115,173,55,28,3,0,2,85,8,180,21,77,142,0,5,6,8,37,0,298,391,721,0,12,7,58,1
5,0A89658C-C739-45CA-9BF1-CBDDDFB922C0,1,4,C13.2,'Malignant neoplasm of posterior wall of hypopha...,1,1949-03-06,0,3,6,34,62,186,78,12,107,36,3081,100,103,84,29,5,3,1,269,17,190,19,14,682,79,6,32,4,36,7,171,108,811,0,17,13,14,1
6,0A89658C-C739-45CA-9BF1-CBDDDFB922C0,2,5,G30,Alzheimer's disease,1,1949-03-06,0,3,6,34,62,186,78,12,107,36,3081,100,103,84,29,5,3,1,269,17,190,19,14,682,79,6,32,4,36,7,171,108,811,0,17,13,14,1
7,0A89658C-C739-45CA-9BF1-CBDDDFB922C0,3,0,H67.2,"'Otitis media in diseases classified elsewhere, ...",1,1949-03-06,0,3,6,34,62,186,78,12,107,36,3081,100,103,84,29,5,3,1,269,17,190,19,14,682,79,6,32,4,36,7,171,108,811,0,17,13,14,1
8,0A89658C-C739-45CA-9BF1-CBDDDFB922C0,4,16,M50.1,Cervical disc disorder with radiculopathy,1,1949-03-06,0,3,6,34,62,186,78,12,107,36,3081,100,103,84,29,5,3,1,269,17,190,19,14,682,79,6,32,4,36,7,171,108,811,0,17,13,14,1
9,0A89658C-C739-45CA-9BF1-CBDDDFB922C0,5,2,C79.11,Secondary malignant neoplasm of bladder,1,1949-03-06,0,3,6,34,62,186,78,12,107,36,3081,100,103,84,29,5,3,1,269,17,190,19,14,682,79,6,32,4,36,7,171,108,811,0,17,13,14,1


# 24) Using different Machine Learning classification algorithms

## 1) LIGHT GBM

In [ ]:
import vaex.ml
from vaex.ml import lightgbm
from lightgbm import LGBMClassifier
from vaex.ml import sklearn 
from vaex.ml.sklearn import Predictor

In [ ]:
features=['PatientGender','PatientMaritalStatus','PatientRace','PatientAgeCategory',
  'CBC: WHITE BLOOD CELL COUNT','CBC: RED BLOOD CELL COUNT', 'CBC: HEMOGLOBIN', 'CBC: HEMATOCRIT',
   'CBC: MEAN CORPUSCULAR VOLUME', 'CBC: MCH', 'CBC: MCHC', 'CBC: RDW',
   'CBC: PLATELET COUNT', 'CBC: ABSOLUTE NEUTROPHILS',
   'CBC: ABSOLUTE LYMPHOCYTES', 'CBC: NEUTROPHILS', 'CBC: LYMPHOCYTES',
   'CBC: MONOCYTES', 'CBC: EOSINOPHILS', 'CBC: BASOPHILS',
   'METABOLIC: SODIUM', 'METABOLIC: POTASSIUM', 'METABOLIC: CHLORIDE',
   'METABOLIC: CARBON DIOXIDE', 'METABOLIC: ANION GAP',
   'METABOLIC: GLUCOSE', 'METABOLIC: BUN', 'METABOLIC: CREATININE',
   'METABOLIC: TOTAL PROTEIN', 'METABOLIC: ALBUMIN', 'METABOLIC: CALCIUM',
   'METABOLIC: BILI TOTAL', 'METABOLIC: AST/SGOT', 'METABOLIC: ALT/SGPT',
   'METABOLIC: ALK PHOS', 'URINALYSIS: SPECIFIC GRAVITY', 'URINALYSIS: PH',
   'URINALYSIS: RED BLOOD CELLS', 'URINALYSIS: WHITE BLOOD CELLS']

target='PrimaryDiagnosisCode'

In [ ]:
LightGB = LGBMClassifier(random_state=0)
vaex_model = Predictor(model=LightGB, features=features, target=target,prediction_name='prediction')
vaex_model.fit(df=data_vaex_train)

data_vaex_train = model.transform(data_vaex_train)
data_vaex_train

KeyboardInterrupt: ignored

## 2) Gradient Boosting

In [ ]:
import vaex.ml
from vaex.ml import sklearn
from vaex.ml.sklearn import Predictor
from sklearn.ensemble import GradientBoostingClassifier

In [ ]:
features=['PatientGender','PatientMaritalStatus','PatientRace','PatientAgeCategory',
  'CBC: WHITE BLOOD CELL COUNT','CBC: RED BLOOD CELL COUNT', 'CBC: HEMOGLOBIN', 'CBC: HEMATOCRIT',
   'CBC: MEAN CORPUSCULAR VOLUME', 'CBC: MCH', 'CBC: MCHC', 'CBC: RDW',
   'CBC: PLATELET COUNT', 'CBC: ABSOLUTE NEUTROPHILS',
   'CBC: ABSOLUTE LYMPHOCYTES', 'CBC: NEUTROPHILS', 'CBC: LYMPHOCYTES',
   'CBC: MONOCYTES', 'CBC: EOSINOPHILS', 'CBC: BASOPHILS',
   'METABOLIC: SODIUM', 'METABOLIC: POTASSIUM', 'METABOLIC: CHLORIDE',
   'METABOLIC: CARBON DIOXIDE', 'METABOLIC: ANION GAP',
   'METABOLIC: GLUCOSE', 'METABOLIC: BUN', 'METABOLIC: CREATININE',
   'METABOLIC: TOTAL PROTEIN', 'METABOLIC: ALBUMIN', 'METABOLIC: CALCIUM',
   'METABOLIC: BILI TOTAL', 'METABOLIC: AST/SGOT', 'METABOLIC: ALT/SGPT',
   'METABOLIC: ALK PHOS', 'URINALYSIS: SPECIFIC GRAVITY', 'URINALYSIS: PH',
   'URINALYSIS: RED BLOOD CELLS', 'URINALYSIS: WHITE BLOOD CELLS']
target='PrimaryDiagnosisCode'

In [ ]:
model_GB=GradientBoostingClassifier(random_state=0)
vaex_model=Predictor(features=features, target=target, model=model_GB, prediction_name='prediction')
vaex_model.fit(df=data_vaex_train)
df = vaex_model.transform(df)
df

## 3) SGDRegressor

In [ ]:
from vaex.ml.sklearn import IncrementalPredictor
vaex.multithreading.thread_count_default = 8
from sklearn.linear_model import SGDRegressor

In [ ]:
features=['PatientGender','PatientMaritalStatus','PatientRace','PatientAgeCategory',
  'CBC: WHITE BLOOD CELL COUNT','CBC: RED BLOOD CELL COUNT', 'CBC: HEMOGLOBIN', 'CBC: HEMATOCRIT',
   'CBC: MEAN CORPUSCULAR VOLUME', 'CBC: MCH', 'CBC: MCHC', 'CBC: RDW',
   'CBC: PLATELET COUNT', 'CBC: ABSOLUTE NEUTROPHILS',
   'CBC: ABSOLUTE LYMPHOCYTES', 'CBC: NEUTROPHILS', 'CBC: LYMPHOCYTES',
   'CBC: MONOCYTES', 'CBC: EOSINOPHILS', 'CBC: BASOPHILS',
   'METABOLIC: SODIUM', 'METABOLIC: POTASSIUM', 'METABOLIC: CHLORIDE',
   'METABOLIC: CARBON DIOXIDE', 'METABOLIC: ANION GAP',
   'METABOLIC: GLUCOSE', 'METABOLIC: BUN', 'METABOLIC: CREATININE',
   'METABOLIC: TOTAL PROTEIN', 'METABOLIC: ALBUMIN', 'METABOLIC: CALCIUM',
   'METABOLIC: BILI TOTAL', 'METABOLIC: AST/SGOT', 'METABOLIC: ALT/SGPT',
   'METABOLIC: ALK PHOS', 'URINALYSIS: SPECIFIC GRAVITY', 'URINALYSIS: PH',
   'URINALYSIS: RED BLOOD CELLS', 'URINALYSIS: WHITE BLOOD CELLS']
target='PrimaryDiagnosisCode'

In [ ]:
model_sgd=SGDRegressor(learning_rate='constant',eta0=0.0001)
vaex_model=IncrementalPredictor(features= features, target=target,model=model_sgd,
                                batch_size=11_000_000,num_epochs=1,shuffle=False)
vaex_model.fit(df=data_vaex_train, progress='widget')
df = vaex_model.transform(df)
df

## 4) CatBoost

In [ ]:
import vaex.ml
from vaex.ml import catboost
from vaex.ml.catboost import CatBoostModel

In [ ]:
features=['PatientGender','PatientMaritalStatus','PatientRace','PatientAgeCategory',
  'CBC: WHITE BLOOD CELL COUNT','CBC: RED BLOOD CELL COUNT', 'CBC: HEMOGLOBIN', 'CBC: HEMATOCRIT',
   'CBC: MEAN CORPUSCULAR VOLUME', 'CBC: MCH', 'CBC: MCHC', 'CBC: RDW',
   'CBC: PLATELET COUNT', 'CBC: ABSOLUTE NEUTROPHILS',
   'CBC: ABSOLUTE LYMPHOCYTES', 'CBC: NEUTROPHILS', 'CBC: LYMPHOCYTES',
   'CBC: MONOCYTES', 'CBC: EOSINOPHILS', 'CBC: BASOPHILS',
   'METABOLIC: SODIUM', 'METABOLIC: POTASSIUM', 'METABOLIC: CHLORIDE',
   'METABOLIC: CARBON DIOXIDE', 'METABOLIC: ANION GAP',
   'METABOLIC: GLUCOSE', 'METABOLIC: BUN', 'METABOLIC: CREATININE',
   'METABOLIC: TOTAL PROTEIN', 'METABOLIC: ALBUMIN', 'METABOLIC: CALCIUM',
   'METABOLIC: BILI TOTAL', 'METABOLIC: AST/SGOT', 'METABOLIC: ALT/SGPT',
   'METABOLIC: ALK PHOS', 'URINALYSIS: SPECIFIC GRAVITY', 'URINALYSIS: PH',
   'URINALYSIS: RED BLOOD CELLS', 'URINALYSIS: WHITE BLOOD CELLS']
target='PrimaryDiagnosisCode'

In [ ]:
parameters = {'learning_rate': 0.1,'max_depth': 1,'random_state': 10}
CatBooster = CatBoostModel(features=features, target=target, num_boost_round=2,params=parameters, prediction_type='Class',prediction_name='catboost_prediction')
CatBooster.fit(df=data_vaex_train, verbose_eval=False)
df_test = CatBooster.transform(data_vaex_train)
df_test

## 5) XGBoost

In [ ]:
import vaex.ml
from vaex.ml import xgboost
from vaex.ml.xgboost import XGBoostModel
import warnings
warnings.filterwarnings('ignore')

In [ ]:
parameters = {'learning_rate': 0.1,'max_depth': 1,
          'random_state': 10,'n_jobs': -1}


XGbooster = XGBoostModel(features=features, target=target, num_boost_round=10, params=parameters)
XGbooster.fit(df=data_vaex_train, evals=[(data_vaex_train, 'train'), (data_vaex_test, 'test')], early_stopping_rounds=5)

data_vaex_test = booster.transform(data_vaex_train)
data_vaex_test

# 25) Final data saved inside the local system

In [ ]:
data.to_csv("/home/admin/ehr_predictivemodeling/Processed_Data/Final_Data_After_Training_Model.csv")

## 26) Load the Model using joblib

In [ ]:
import joblib
joblib.dump(model_rf,'EHR_Trained_Model.ml')

In [ ]:
model_rf=joblib.load('EHR_Trained_Model.ml')  